In [47]:
import numpy as np
import argparse
import os
import imp
import re
import pickle5 as pickle
import datetime
import random
import math
import logging
import copy
import matplotlib.pyplot as plt
import sklearn
import logging
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import kneighbors_graph

from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn import Parameter

from utils import utils
from utils.readers import InHospitalMortalityReader
from utils.preprocessing import Discretizer, Normalizer
from utils import metrics
from utils import common_utils

In [48]:
# Select the target dataset: COVID-19 Dataset from TJ Hospital or HM Hospital
target_dataset = 'HM' # 'TJ' or 'HM'

# Use CUDA if available
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
print("available device: {}".format(device))

available device: cuda:0


In [49]:
def get_logger(name):
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    
    # 以下两行是为了在jupyter notebook 中不重复输出日志
    if logger.root.handlers:
        logger.root.handlers[0].setLevel(logging.WARNING)
 
    handler_stdout = logging.StreamHandler()
    handler_stdout.setLevel(logging.INFO)
    handler_stdout.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
    logger.addHandler(handler_stdout)
 
    handler_file = logging.FileHandler('log_file.log', encoding='utf-8')
    handler_file.setLevel(logging.DEBUG)
    handler_file.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
    logger.addHandler(handler_file)
 
    return logger

logger = get_logger(__name__)

logger.debug('这是希望输出的debug内容')
logger.info('这是希望输出的info内容')
logger.warning('这是希望输出的warning内容')

2023-08-06 12:56:22,481 - INFO - 这是希望输出的info内容
2023-08-06 12:56:22,481 - INFO - 这是希望输出的info内容
2023-08-06 12:56:22,483 - WARNING - 这是希望输出的warning内容
2023-08-06 12:56:22,483 - WARNING - 这是希望输出的warning内容


### Transfer Source Data & Model: 
#### Teacher Model


In [50]:
data_path = './data/Challenge/'
small_part = False
arg_timestep = 1.0
batch_size = 256
epochs = 100

In [51]:
all_x = pickle.load(open(data_path + 'new_x_front_fill.dat', 'rb'))
all_y = pickle.load(open(data_path + 'new_y_front_fill.dat', 'rb'))
all_names = pickle.load(open(data_path + 'new_name.dat', 'rb'))
static = pickle.load(open(data_path + 'new_demo_front_fill.dat', 'rb'))
mask_x = pickle.load(open(data_path + 'new_mask_x.dat', 'rb'))
mask_demo = pickle.load(open(data_path + 'new_mask_demo.dat', 'rb'))
all_x_len = [len(i) for i in all_x]

print(all_x[0])
print(mask_x[0])
print(all_names[0])
print(static[0])
logger.info(all_x[0])
logger.info(mask_x[0])
logger.info(all_names[0])
logger.info(static[0])

2023-08-06 12:56:52,728 - INFO - [[-0.06242012453646045, 0.2744523712853132, 0.02957319402431667, -0.11839355366098099, -0.14686926072725967, -0.1311661871017867, -0.1425010869914041, 0.005325137533142886, 0.16066034068876195, -0.01724690479013476, -0.004930129148398766, 0.014295447077977357, -0.11026740609348425, 0.39895882324044557, -0.2561829490343818, -0.34587251014597875, -0.2371260510881844, 0.3051487380880145, 0.029264930048844468, -0.3160730875843661, -0.3766591890459441, -0.1935716453287135, -0.3351561343174943, -0.12930577851172065, -0.17160263114220592, -0.05532679134287014, -0.2815944741293343, -0.32211134163582006, -0.0899704549996704, -0.06645805008034258, -0.33684497792590695, -0.14828727498338712, -0.24435830491350327, -0.14487324959363315], [-0.06242012453646045, 0.2744523712853132, 0.02957319402431667, -0.11839355366098099, -0.14686926072725967, -0.1311661871017867, -0.1425010869914041, 0.005325137533142886, 0.16066034068876195, -0.01724690479013476, -0.00493012914839

2023-08-06 12:56:52,728 - INFO - [[-0.06242012453646045, 0.2744523712853132, 0.02957319402431667, -0.11839355366098099, -0.14686926072725967, -0.1311661871017867, -0.1425010869914041, 0.005325137533142886, 0.16066034068876195, -0.01724690479013476, -0.004930129148398766, 0.014295447077977357, -0.11026740609348425, 0.39895882324044557, -0.2561829490343818, -0.34587251014597875, -0.2371260510881844, 0.3051487380880145, 0.029264930048844468, -0.3160730875843661, -0.3766591890459441, -0.1935716453287135, -0.3351561343174943, -0.12930577851172065, -0.17160263114220592, -0.05532679134287014, -0.2815944741293343, -0.32211134163582006, -0.0899704549996704, -0.06645805008034258, -0.33684497792590695, -0.14828727498338712, -0.24435830491350327, -0.14487324959363315], [-0.06242012453646045, 0.2744523712853132, 0.02957319402431667, -0.11839355366098099, -0.14686926072725967, -0.1311661871017867, -0.1425010869914041, 0.005325137533142886, 0.16066034068876195, -0.01724690479013476, -0.00493012914839

2023-08-06 12:56:52,740 - INFO - [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0,

2023-08-06 12:56:52,743 - INFO - 110609
2023-08-06 12:56:52,743 - INFO - 110609
2023-08-06 12:56:52,746 - INFO - [[-1.1264803265802585, -0.9931656033414253, 0.9931656033414252, 0.11854736004896246, -0.8962118618028821, '0'], [-1.1264803265802585, -0.9931656033414253, 0.9931656033414252, 0.11854736004896246, -0.8617355345389544, '0'], [-1.1264803265802585, -0.9931656033414253, 0.9931656033414252, 0.11854736004896246, -0.8272592072750267, '0'], [-1.1264803265802585, -0.9931656033414253, 0.9931656033414252, 0.11854736004896246, -0.7927828800110989, '0'], [-1.1264803265802585, -0.9931656033414253, 0.9931656033414252, 0.11854736004896246, -0.7583065527471711, '0'], [-1.1264803265802585, -0.9931656033414253, 0.9931656033414252, 0.11854736004896246, -0.7238302254832434, '0'], [-1.1264803265802585, -0.9931656033414253, 0.9931656033414252, 0.11854736004896246, -0.6893538982193156, '0'], [-1.1264803265802585, -0.9931656033414253, 0.9931656033414252, 0.11854736004896246, -0.6548775709553878, '0']

[[-0.06242012453646045, 0.2744523712853132, 0.02957319402431667, -0.11839355366098099, -0.14686926072725967, -0.1311661871017867, -0.1425010869914041, 0.005325137533142886, 0.16066034068876195, -0.01724690479013476, -0.004930129148398766, 0.014295447077977357, -0.11026740609348425, 0.39895882324044557, -0.2561829490343818, -0.34587251014597875, -0.2371260510881844, 0.3051487380880145, 0.029264930048844468, -0.3160730875843661, -0.3766591890459441, -0.1935716453287135, -0.3351561343174943, -0.12930577851172065, -0.17160263114220592, -0.05532679134287014, -0.2815944741293343, -0.32211134163582006, -0.0899704549996704, -0.06645805008034258, -0.33684497792590695, -0.14828727498338712, -0.24435830491350327, -0.14487324959363315], [-0.06242012453646045, 0.2744523712853132, 0.02957319402431667, -0.11839355366098099, -0.14686926072725967, -0.1311661871017867, -0.1425010869914041, 0.005325137533142886, 0.16066034068876195, -0.01724690479013476, -0.004930129148398766, 0.014295447077977357, -0.11

In [52]:
if target_dataset == 'TJ':
    subset_idx = [27, 29, 18, 16, 26, 33, 28, 31, 32, 15, 11, 25, 21, 20, 9, 17, 30, 19]
elif target_dataset == 'HM':
    subset_idx = [0, 1, 2, 3, 5, 9, 11, 12, 13, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]

subset_cnt = len(subset_idx)
other_idx = []
for i in range(len(all_x[0][0])):
    if i not in subset_idx:
        other_idx.append(i)

for i in range(len(all_x)):
    cur = np.array(all_x[i], dtype=float)
    cur_mask = np.array(mask_x[i])
    cur_subset = cur[:, subset_idx]
    cur_other = cur[:, other_idx]
    cur_mask_subset = cur_mask[:, subset_idx]
    cur_mask_other = cur_mask[:, other_idx]
    all_x[i] = np.concatenate((cur_subset, cur_other), axis=1).tolist()
    mask_x[i] = np.concatenate((cur_mask_subset, cur_mask_other), axis=1).tolist()
print(all_x[0])
print(mask_x[0])
print(len(all_x[0][0]))
logger.info(all_x[0])
logger.info(mask_x[0])
logger.info(len(all_x[0][0]))

2023-08-06 12:57:09,369 - INFO - [[-0.06242012453646045, 0.2744523712853132, 0.02957319402431667, -0.11839355366098099, -0.1311661871017867, -0.01724690479013476, 0.014295447077977357, -0.11026740609348425, 0.39895882324044557, 0.3051487380880145, 0.029264930048844468, -0.3160730875843661, -0.3766591890459441, -0.1935716453287135, -0.3351561343174943, -0.12930577851172065, -0.17160263114220592, -0.05532679134287014, -0.2815944741293343, -0.32211134163582006, -0.0899704549996704, -0.06645805008034258, -0.33684497792590695, -0.14828727498338712, -0.24435830491350327, -0.14487324959363315, -0.14686926072725967, -0.1425010869914041, 0.005325137533142886, 0.16066034068876195, -0.004930129148398766, -0.2561829490343818, -0.34587251014597875, -0.2371260510881844], [-0.06242012453646045, 0.2744523712853132, 0.02957319402431667, -0.11839355366098099, -0.1311661871017867, -0.01724690479013476, 0.014295447077977357, -0.11026740609348425, 0.39895882324044557, 0.3051487380880145, 0.0292649300488444

2023-08-06 12:57:09,369 - INFO - [[-0.06242012453646045, 0.2744523712853132, 0.02957319402431667, -0.11839355366098099, -0.1311661871017867, -0.01724690479013476, 0.014295447077977357, -0.11026740609348425, 0.39895882324044557, 0.3051487380880145, 0.029264930048844468, -0.3160730875843661, -0.3766591890459441, -0.1935716453287135, -0.3351561343174943, -0.12930577851172065, -0.17160263114220592, -0.05532679134287014, -0.2815944741293343, -0.32211134163582006, -0.0899704549996704, -0.06645805008034258, -0.33684497792590695, -0.14828727498338712, -0.24435830491350327, -0.14487324959363315, -0.14686926072725967, -0.1425010869914041, 0.005325137533142886, 0.16066034068876195, -0.004930129148398766, -0.2561829490343818, -0.34587251014597875, -0.2371260510881844], [-0.06242012453646045, 0.2744523712853132, 0.02957319402431667, -0.11839355366098099, -0.1311661871017867, -0.01724690479013476, 0.014295447077977357, -0.11026740609348425, 0.39895882324044557, 0.3051487380880145, 0.0292649300488444

2023-08-06 12:57:09,377 - INFO - [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0], [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0, 0, 0,

2023-08-06 12:57:09,379 - INFO - 34
2023-08-06 12:57:09,379 - INFO - 34


[[-0.06242012453646045, 0.2744523712853132, 0.02957319402431667, -0.11839355366098099, -0.1311661871017867, -0.01724690479013476, 0.014295447077977357, -0.11026740609348425, 0.39895882324044557, 0.3051487380880145, 0.029264930048844468, -0.3160730875843661, -0.3766591890459441, -0.1935716453287135, -0.3351561343174943, -0.12930577851172065, -0.17160263114220592, -0.05532679134287014, -0.2815944741293343, -0.32211134163582006, -0.0899704549996704, -0.06645805008034258, -0.33684497792590695, -0.14828727498338712, -0.24435830491350327, -0.14487324959363315, -0.14686926072725967, -0.1425010869914041, 0.005325137533142886, 0.16066034068876195, -0.004930129148398766, -0.2561829490343818, -0.34587251014597875, -0.2371260510881844], [-0.06242012453646045, 0.2744523712853132, 0.02957319402431667, -0.11839355366098099, -0.1311661871017867, -0.01724690479013476, 0.014295447077977357, -0.11026740609348425, 0.39895882324044557, 0.3051487380880145, 0.029264930048844468, -0.3160730875843661, -0.37665

In [53]:
train_num =int( len(all_x) * 0.8) + 1
print(train_num)
logger.info(train_num)
dev_num =int( len(all_x) * 0.1) + 1
print(dev_num)
logger.info(dev_num)
test_num =int( len(all_x) * 0.1)
print(test_num)
logger.info(test_num)
assert(train_num+dev_num+test_num == len(all_x))

train_x = []
train_y = []
train_names = []
train_static = []
train_x_len = []
train_mask_x = []
for idx in range(train_num):
    train_x.append(all_x[idx])
    train_y.append(int(all_y[idx][-1]))
    train_names.append(all_names[idx])
    train_static.append(static[idx])
    train_x_len.append(all_x_len[idx])
    train_mask_x.append(mask_x[idx])

dev_x = []
dev_y = []
dev_names = []
dev_static = []
dev_x_len = []
dev_mask_x = []
for idx in range(train_num, train_num + dev_num):
    dev_x.append(all_x[idx])
    dev_y.append(int(all_y[idx][-1]))
    dev_names.append(all_names[idx])
    dev_static.append(static[idx])
    dev_x_len.append(all_x_len[idx])
    dev_mask_x.append(mask_x[idx])


test_x = []
test_y = []
test_names = []
test_static = []
test_x_len = []
test_mask_x = []
for idx in range(train_num + dev_num, train_num + dev_num + test_num):
    test_x.append(all_x[idx])
    test_y.append(int(all_y[idx][-1]))
    test_names.append(all_names[idx])
    test_static.append(static[idx])
    test_x_len.append(all_x_len[idx])
    test_mask_x.append(mask_x[idx])


assert(len(train_x) == train_num)
assert(len(dev_x) == dev_num)
assert(len(test_x) == test_num)

32269


2023-08-06 12:57:11,278 - INFO - 32269
2023-08-06 12:57:11,278 - INFO - 32269
2023-08-06 12:57:11,280 - INFO - 4034
2023-08-06 12:57:11,280 - INFO - 4034
2023-08-06 12:57:11,281 - INFO - 4033
2023-08-06 12:57:11,281 - INFO - 4033


4034
4033


In [54]:
long_x = all_x
long_y = [y[-1] for y in all_y]


In [55]:
def get_loss(y_pred, y_true):
    loss = torch.nn.BCELoss()
    return loss(y_pred, y_true)

In [56]:
def get_re_loss(y_pred, y_true):
    loss = torch.nn.MSELoss()
    return loss(y_pred, y_true)

In [57]:
def get_kl_loss(x_pred, x_target):
    loss = torch.nn.KLDivLoss(reduce=True, size_average=True)
    return loss(x_pred, x_target)

In [58]:
def get_wass_dist(x_pred, x_target):
    m1 = torch.mean(x_pred, dim=0)
    m2 = torch.mean(x_target, dim=0)
    v1 = torch.var(x_pred, dim=0)
    v2 = torch.var(x_target, dim=0)
    p1 = torch.sum(torch.pow((m1 - m2), 2))
    p2 = torch.sum(torch.pow(torch.pow(v1, 1/2) - torch.pow(v2, 1/2), 2))
    return torch.pow(p1+p2, 1/2)

In [59]:
def pad_sents(sents, pad_token):

    sents_padded = []

    max_length = max([len(_) for _ in sents])
    for i in sents:
        padded = list(i) + [pad_token]*(max_length-len(i))
        sents_padded.append(np.array(padded))


    return np.array(sents_padded)

In [60]:
def batch_iter(x, y, mask, lens, batch_size, shuffle=False):
    """ Yield batches of source and target sentences reverse sorted by length (largest to smallest).
    @param data (list of (src_sent, tgt_sent)): list of tuples containing source and target sentence
    @param batch_size (int): batch size
    @param shuffle (boolean): whether to randomly shuffle the dataset
    """
    batch_num = math.ceil(len(x) / batch_size) # 向下取整
    index_array = list(range(len(x)))

    if shuffle:
        np.random.shuffle(index_array)

    for i in range(batch_num):
        indices = index_array[i * batch_size: (i + 1) * batch_size] #  fetch out all the induces
        
        examples = []
        for idx in indices:
            examples.append((x[idx], y[idx], mask[idx], lens[idx]))
       
        examples = sorted(examples, key=lambda e: len(e[0]), reverse=True)
    
        batch_x = [e[0] for e in examples]
        batch_y = [e[1] for e in examples]
        batch_mask_x = [e[2] for e in examples]
#         batch_name = [e[2] for e in examples]
        batch_lens = [e[3] for e in examples]

        yield batch_x, batch_y, batch_mask_x, batch_lens

In [61]:
def length_to_mask(length, max_len=None, dtype=None):
    """length: B.
    return B x max_len.
    If max_len is None, then max of length will be used.
    """
    assert len(length.shape) == 1, 'Length shape should be 1 dimensional.'
    max_len = max_len or length.max().item()
    mask = torch.arange(max_len, device=length.device,
                        dtype=length.dtype).expand(len(length), max_len) < length.unsqueeze(1)
    if dtype is not None:
        mask = torch.as_tensor(mask, dtype=dtype, device=length.device)
    return mask

In [62]:
class SingleAttention(nn.Module):
    def __init__(self, attention_input_dim, attention_hidden_dim, attention_type='add', demographic_dim=12, time_aware=False, use_demographic=False):
        super(SingleAttention, self).__init__()
        
        self.attention_type = attention_type
        self.attention_hidden_dim = attention_hidden_dim
        self.attention_input_dim = attention_input_dim
        self.use_demographic = use_demographic
        self.demographic_dim = demographic_dim
        self.time_aware = time_aware

        # batch_time = torch.arange(0, batch_mask.size()[1], dtype=torch.float32).reshape(1, batch_mask.size()[1], 1)
        # batch_time = batch_time.repeat(batch_mask.size()[0], 1, 1)
        
        if attention_type == 'add':
            if self.time_aware == True:
                # self.Wx = nn.Parameter(torch.randn(attention_input_dim+1, attention_hidden_dim))
                self.Wx = nn.Parameter(torch.randn(attention_input_dim, attention_hidden_dim))
                self.Wtime_aware = nn.Parameter(torch.randn(1, attention_hidden_dim))
                nn.init.kaiming_uniform_(self.Wtime_aware, a=math.sqrt(5))
            else:
                self.Wx = nn.Parameter(torch.randn(attention_input_dim, attention_hidden_dim))
            self.Wt = nn.Parameter(torch.randn(attention_input_dim, attention_hidden_dim))
            self.Wd = nn.Parameter(torch.randn(demographic_dim, attention_hidden_dim))
            self.bh = nn.Parameter(torch.zeros(attention_hidden_dim,))
            self.Wa = nn.Parameter(torch.randn(attention_hidden_dim, 1))
            self.ba = nn.Parameter(torch.zeros(1,))
            
            nn.init.kaiming_uniform_(self.Wd, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wx, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wt, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        elif attention_type == 'mul':
            self.Wa = nn.Parameter(torch.randn(attention_input_dim, attention_input_dim))
            self.ba = nn.Parameter(torch.zeros(1,))
            
            nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        elif attention_type == 'concat':
            if self.time_aware == True:
                self.Wh = nn.Parameter(torch.randn(2*attention_input_dim+1, attention_hidden_dim))
            else:
                self.Wh = nn.Parameter(torch.randn(2*attention_input_dim, attention_hidden_dim))

            self.Wa = nn.Parameter(torch.randn(attention_hidden_dim, 1))
            self.ba = nn.Parameter(torch.zeros(1,))
            
            nn.init.kaiming_uniform_(self.Wh, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        else:
            raise RuntimeError('Wrong attention type.')
        
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax()
    
    def forward(self, input, demo=None):
 
        batch_size, time_step, input_dim = input.size() # batch_size * time_step * hidden_dim(i)
        time_decays = torch.tensor(range(time_step-1,-1,-1), dtype=torch.float32).unsqueeze(-1).unsqueeze(0).to(device)# 1*t*1
        b_time_decays = time_decays.repeat(batch_size,1,1)# b t 1
        
        if self.attention_type == 'add': #B*T*I  @ H*I
            q = torch.matmul(input[:,-1,:], self.Wt)# b h
            q = torch.reshape(q, (batch_size, 1, self.attention_hidden_dim)) #B*1*H
            if self.time_aware == True:
                # k_input = torch.cat((input, time), dim=-1)
                k = torch.matmul(input, self.Wx)#b t h
                # k = torch.reshape(k, (batch_size, 1, time_step, self.attention_hidden_dim)) #B*1*T*H
                time_hidden = torch.matmul(b_time_decays, self.Wtime_aware)#  b t h
            else:
                k = torch.matmul(input, self.Wx)# b t h
                # k = torch.reshape(k, (batch_size, 1, time_step, self.attention_hidden_dim)) #B*1*T*H
            if self.use_demographic == True:
                d = torch.matmul(demo, self.Wd) #B*H
                d = torch.reshape(d, (batch_size, 1, self.attention_hidden_dim)) # b 1 h
            h = q + k + self.bh # b t h
            if self.time_aware == True:
                h += time_hidden
            h = self.tanh(h) #B*T*H
            e = torch.matmul(h, self.Wa) + self.ba #B*T*1
            e = torch.reshape(e, (batch_size, time_step))# b t
        elif self.attention_type == 'mul':
            e = torch.matmul(input[:,-1,:], self.Wa)#b i
            e = torch.matmul(e.unsqueeze(1), input.permute(0,2,1)).squeeze() + self.ba #b t
        elif self.attention_type == 'concat':
            q = input[:,-1,:].unsqueeze(1).repeat(1,time_step,1)# b t i
            k = input
            c = torch.cat((q, k), dim=-1) #B*T*2I
            if self.time_aware == True:
                c = torch.cat((c, b_time_decays), dim=-1) #B*T*2I+1
            h = torch.matmul(c, self.Wh)
            h = self.tanh(h)
            e = torch.matmul(h, self.Wa) + self.ba #B*T*1
            e = torch.reshape(e, (batch_size, time_step)) # b t 
        
        a = self.softmax(e) #B*T
        v = torch.matmul(a.unsqueeze(1), input).squeeze() #B*I

        return v, a

class FinalAttentionQKV(nn.Module):
    def __init__(self, attention_input_dim, attention_hidden_dim, attention_type='add', dropout=None):
        super(FinalAttentionQKV, self).__init__()
        
        self.attention_type = attention_type
        self.attention_hidden_dim = attention_hidden_dim
        self.attention_input_dim = attention_input_dim


        self.W_q = nn.Linear(attention_input_dim, attention_hidden_dim)
        self.W_k = nn.Linear(attention_input_dim, attention_hidden_dim)
        self.W_v = nn.Linear(attention_input_dim, attention_hidden_dim)

        self.W_out = nn.Linear(attention_hidden_dim, 1)

        self.b_in = nn.Parameter(torch.zeros(1,))
        self.b_out = nn.Parameter(torch.zeros(1,))

        nn.init.kaiming_uniform_(self.W_q.weight, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.W_k.weight, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.W_v.weight, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.W_out.weight, a=math.sqrt(5))

        self.Wh = nn.Parameter(torch.randn(2*attention_input_dim, attention_hidden_dim))
        self.Wa = nn.Parameter(torch.randn(attention_hidden_dim, 1))
        self.ba = nn.Parameter(torch.zeros(1,))
        
        nn.init.kaiming_uniform_(self.Wh, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        
        self.dropout = nn.Dropout(p=dropout)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input):
 
        batch_size, time_step, input_dim = input.size() # batch_size * input_dim + 1 * hidden_dim(i)
        input_q = self.W_q(torch.mean(input, dim=1)) # b h
        input_k = self.W_k(input)# b t h
        input_v = self.W_v(input)# b t h

        if self.attention_type == 'add': #B*T*I  @ H*I

            q = torch.reshape(input_q, (batch_size, 1, self.attention_hidden_dim)) #B*1*H
            h = q + input_k + self.b_in # b t h
            h = self.tanh(h) #B*T*H
            e = self.W_out(h) # b t 1
            e = torch.reshape(e, (batch_size, time_step))# b t

        elif self.attention_type == 'mul':
            q = torch.reshape(input_q, (batch_size, self.attention_hidden_dim, 1)) #B*h 1
            e = torch.matmul(input_k, q).squeeze()#b t
            
        elif self.attention_type == 'concat':
            q = input_q.unsqueeze(1).repeat(1,time_step,1)# b t h
            k = input_k
            c = torch.cat((q, k), dim=-1) #B*T*2I
            h = torch.matmul(c, self.Wh)
            h = self.tanh(h)
            e = torch.matmul(h, self.Wa) + self.ba #B*T*1
            e = torch.reshape(e, (batch_size, time_step)) # b t 
        
        a = self.softmax(e) #B*T
        if self.dropout is not None:
            a = self.dropout(a)
        v = torch.matmul(a.unsqueeze(1), input_v).squeeze() #B*I

        return v, a

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

def tile(a, dim, n_tile):
    init_dim = a.size(dim)
    repeat_idx = [1] * a.dim()
    repeat_idx[dim] = n_tile
    a = a.repeat(*(repeat_idx))
    order_index = torch.LongTensor(np.concatenate([init_dim * np.arange(n_tile) + i for i in range(init_dim)])).to(device)
    return torch.index_select(a, dim, order_index).to(device)

class PositionwiseFeedForward(nn.Module): # new added
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x)))), None

    
class PositionalEncoding(nn.Module): # new added / not use anymore
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=400):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0., max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0 

def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)# b h t d_k
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k) # b h t t
    if mask is not None:# 1 1 t t
        scores = scores.masked_fill(mask == 0, -1e9)# b h t t 
    p_attn = F.softmax(scores, dim = -1)# b h t t
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn # b h t v (d_k) 
    
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, self.d_k * self.h), 3)
        self.final_linear = nn.Linear(d_model, d_model)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1) # 1 1 t t

        nbatches = query.size(0)# b
        input_dim = query.size(1)# i+1
        feature_dim = query.size(-1)# i+1

        #input size -> # batch_size * d_input * hidden_dim
        
        # d_model => h * d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))] # b num_head d_input d_k
        
       
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)# b num_head d_input d_v (d_k) 
        
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)# batch_size * d_input * hidden_dim

        #DeCov 
        DeCov_contexts = x.transpose(0, 1).transpose(1, 2) # I+1 H B
#         print(DeCov_contexts.shape)
        Covs = cov(DeCov_contexts[0,:,:])
        DeCov_loss = 0.5 * (torch.norm(Covs, p = 'fro')**2 - torch.norm(torch.diag(Covs))**2 ) 
        for i in range(11 -1):
            Covs = cov(DeCov_contexts[i+1,:,:])
            DeCov_loss += 0.5 * (torch.norm(Covs, p = 'fro')**2 - torch.norm(torch.diag(Covs))**2 ) 


        return self.final_linear(x), DeCov_loss

class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-7):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

def cov(m, y=None):
    if y is not None:
        m = torch.cat((m, y), dim=0)
    m_exp = torch.mean(m, dim=1)
    x = m - m_exp[:, None]
    cov = 1 / (x.size(1) - 1) * x.mm(x.t())
    return cov

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        returned_value = sublayer(self.norm(x))
        return x + self.dropout(returned_value[0]) , returned_value[1]

class distcare_teacher(nn.Module):
    def __init__(self, input_dim, hidden_dim, d_model,  MHD_num_head, d_ff, output_dim, keep_prob=0.5):
        super(distcare_teacher, self).__init__()

        # hyperparameters
        self.input_dim = input_dim  
        self.hidden_dim = hidden_dim  # d_model
        self.d_model = d_model
        self.MHD_num_head = MHD_num_head
        self.d_ff = d_ff
        self.output_dim = output_dim
        self.keep_prob = keep_prob

        # layers
        self.PositionalEncoding = PositionalEncoding(self.d_model, dropout = 0, max_len = 400)

        self.GRUs = clones(nn.GRU(1, self.hidden_dim, batch_first = True), self.input_dim)
        self.LastStepAttentions = clones(SingleAttention(self.hidden_dim, 8, attention_type='concat', demographic_dim=12, time_aware=True, use_demographic=False),self.input_dim)
        
        self.FinalAttentionQKV = FinalAttentionQKV(self.hidden_dim, self.hidden_dim, attention_type='mul',dropout = 1 - self.keep_prob)

        self.MultiHeadedAttention = MultiHeadedAttention(self.MHD_num_head, self.d_model,dropout = 1 - self.keep_prob)
        self.SublayerConnection = SublayerConnection(self.d_model, dropout = 1 - self.keep_prob)

        self.PositionwiseFeedForward = PositionwiseFeedForward(self.d_model, self.d_ff, dropout=0.1)

        self.demo_proj_main = nn.Linear(12, self.hidden_dim)
        self.demo_proj = nn.Linear(12, self.hidden_dim)
        self.output = nn.Linear(self.hidden_dim, self.output_dim)

        self.dropout = nn.Dropout(p = 1 - self.keep_prob)
        self.tanh=nn.Tanh()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()

    def forward(self, input, lens):
        lens = lens.to('cpu')
        # input shape [batch_size, timestep, feature_dim]
#         demo_main = self.tanh(self.demo_proj_main(demo_input)).unsqueeze(1)# b hidden_dim
        
        batch_size = input.size(0)
        time_step = input.size(1)
        feature_dim = input.size(2)
        assert(feature_dim == self.input_dim)# input Tensor : 256 * 48 * 76
        assert(self.d_model % self.MHD_num_head == 0)

        
        GRU_embeded_input = self.GRUs[0](pack_padded_sequence(input[:,:,0].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
#         print(GRU_embeded_input.shape)
        for i in range(feature_dim-1):
            embeded_input = self.GRUs[i+1](pack_padded_sequence(input[:,:,i+1].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
            GRU_embeded_input = torch.cat((GRU_embeded_input, embeded_input), 1)

#         GRU_embeded_input = torch.cat((GRU_embeded_input, demo_main), 1)# b i+1 h
        posi_input = self.dropout(GRU_embeded_input) # batch_size * d_input * hidden_dim


        #mask = subsequent_mask(time_step).to(device) # 1 t t 
        contexts = self.SublayerConnection(posi_input, lambda x: self.MultiHeadedAttention(posi_input, posi_input, posi_input, None))# # batch_size * d_input * hidden_dim
    
        DeCov_loss = contexts[1]
        contexts = contexts[0]

        contexts = self.SublayerConnection(contexts, lambda x: self.PositionwiseFeedForward(contexts))[0]# # batch_size * d_input * hidden_dim
        #contexts = contexts.view(batch_size, feature_dim * self.hidden_dim)#
        # contexts = torch.matmul(self.Wproj, contexts) + self.bproj
        # contexts = contexts.squeeze()
        # demo_key = self.demo_proj(demo_input)# b hidden_dim
        # demo_key = self.relu(demo_key)
        # input_dim_scores = torch.matmul(contexts, demo_key.unsqueeze(-1)).squeeze() # b i
        # input_dim_scores = self.dropout(self.sigmoid(input_dim_scores)).unsqueeze(1)# b i
        
        # weighted_contexts = torch.matmul(input_dim_scores, contexts).squeeze()
#         print(contexts.shape)

        weighted_contexts = self.FinalAttentionQKV(contexts)[0]
        output = self.output(self.dropout(weighted_contexts))# b 1
        output = self.sigmoid(output)
#         print(weighted_contexts.shape)
          
        return output, DeCov_loss  ,weighted_contexts
    #, self.MultiHeadedAttention.attn




In [63]:
RANDOM_SEED = 43
np.random.seed(RANDOM_SEED) #numpy
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED) # cpu
torch.cuda.manual_seed(RANDOM_SEED) #gpu
torch.backends.cudnn.deterministic=True # cudnn
    
epochs = 150
batch_size = 256
input_dim = 34
hidden_dim = 32
d_model = 32
MHD_num_head = 4
d_ff = 64
output_dim = 1

model = distcare_teacher(input_dim = input_dim, hidden_dim = hidden_dim, d_model=d_model, MHD_num_head=MHD_num_head, d_ff=d_ff, output_dim = output_dim).to(device)
# input_dim, d_model, d_k, d_v, MHD_num_head, d_ff, output_dim
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# Training Teacher
# If you don't want to train Teacher Model:
# - The pretrained taecher model is in direcrtory './model/', and can be directly loaded, 
# - Simply skip this cell and load the model to validate on Dev Dataset.
logger.info('Training Teacher')

total_train_loss = []
total_valid_loss = []
global_best = 0
auroc = []
auprc = []
minpse = []
history = []

pad_token = np.zeros(input_dim)
# begin_time = time.time()
best_auroc = 0
best_auprc = 0
best_minpse = 0
    
if target_dataset == 'TJ':    
    file_name = './model/pretrained-challenge-front-fill-teacher-2covid'
elif target_dataset == 'HM':
    file_name = './model/pretrained-challenge-front-fill-teacher-2spain'

for each_epoch in range(epochs):

    epoch_loss = []
    counter_batch = 0
    model.train()  

    for step, (batch_x, batch_y, batch_mask_x, batch_lens) in enumerate(batch_iter(train_x, train_y, train_mask_x, train_x_len, batch_size, shuffle=True)):  
        optimizer.zero_grad()
        batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
        batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
        batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
        batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)

#        masks = length_to_mask(batch_lens).unsqueeze(-1).float()

        opt, decov_loss, emb = model(batch_x, batch_lens)

        BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1)) # b t 1
#             REC_Loss = F.mse_loss(masks * recon, masks * batch_x, reduction='mean').to(device)

        loss = BCE_Loss #+ 1000 * decov_loss

        epoch_loss.append(BCE_Loss.cpu().detach().numpy())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 20)
        optimizer.step()

        if step % 20 == 0:
            print('Epoch %d Batch %d: Train Loss = %.4f'%(each_epoch, step, loss.cpu().detach().numpy()))
            logger.info('Epoch %d Batch %d: Train Loss = %.4f'%(each_epoch, step, loss.cpu().detach().numpy()))

    epoch_loss = np.mean(epoch_loss)
    total_train_loss.append(epoch_loss)

    #Validation
    y_true = []
    y_pred = []
    with torch.no_grad():
        model.eval()
        valid_loss = []
        valid_true = []
        valid_pred = []
        for batch_x, batch_y, batch_mask_x, batch_lens in batch_iter(dev_x, dev_y, dev_mask_x, dev_x_len, batch_size):
            batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
            batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
            batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
            batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)
#            masks = length_to_mask(batch_lens).unsqueeze(-1).float()


            opt, decov_loss, emb = model(batch_x, batch_lens)

            BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))
#                 REC_Loss = F.mse_loss(recon, batch_x, reduction='mean').to(device)

            valid_loss.append(BCE_Loss.cpu().detach().numpy())

            y_pred += list(opt.cpu().detach().numpy().flatten())
            y_true += list(batch_y.cpu().numpy().flatten())

        valid_loss = np.mean(valid_loss)
        total_valid_loss.append(valid_loss)
        ret = metrics.print_metrics_binary(y_true, y_pred,verbose = 0)
        history.append(ret)
        #print()

        print('Epoch %d: Loss = %.4f Valid loss = %.4f roc = %.4f'%(each_epoch, total_train_loss[-1], total_valid_loss[-1], ret['auroc']))
        logger.info('Epoch %d: Loss = %.4f Valid loss = %.4f roc = %.4f'%(each_epoch, total_train_loss[-1], total_valid_loss[-1], ret['auroc']))
        metrics.print_metrics_binary(y_true, y_pred)

        cur_auroc = ret['auroc']
        if cur_auroc > best_auroc:
            best_auroc = cur_auroc
            best_auprc = ret['auprc']
            best_minpse = ret['minpse']
            state = {
                'net': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': each_epoch
            }
            torch.save(state, file_name)
            print('------------ Save best model - AUROC: %.4f ------------'%cur_auroc)       

print('auroc %.4f'%(best_auroc))
print('auprc %.4f'%(best_auprc))
print('minpse %.4f'%(best_minpse))  
logger.info('auroc %.4f'%(best_auroc))
logger.info('auprc %.4f'%(best_auprc))
logger.info('minpse %.4f'%(best_minpse))

2023-03-23 18:27:15,845 - INFO - Training Teacher
2023-03-23 18:27:15,845 - INFO - Training Teacher
2023-03-23 18:27:15,845 - INFO - Training Teacher
/home/zzj/anaconda3/envs/distcare/lib/python3.7/site-packages/ipykernel_launcher.py:163: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
2023-03-23 18:27:16,408 - INFO - Epoch 0 Batch 0: Train Loss = 0.2132
2023-03-23 18:27:16,408 - INFO - Epoch 0 Batch 0: Train Loss = 0.2132
2023-03-23 18:27:16,408 - INFO - Epoch 0 Batch 0: Train Loss = 0.2132


Epoch 0 Batch 0: Train Loss = 0.2132


2023-03-23 18:27:26,649 - INFO - Epoch 0 Batch 20: Train Loss = 0.2136
2023-03-23 18:27:26,649 - INFO - Epoch 0 Batch 20: Train Loss = 0.2136
2023-03-23 18:27:26,649 - INFO - Epoch 0 Batch 20: Train Loss = 0.2136


Epoch 0 Batch 20: Train Loss = 0.2136


2023-03-23 18:27:36,981 - INFO - Epoch 0 Batch 40: Train Loss = 0.2707
2023-03-23 18:27:36,981 - INFO - Epoch 0 Batch 40: Train Loss = 0.2707
2023-03-23 18:27:36,981 - INFO - Epoch 0 Batch 40: Train Loss = 0.2707


...

Epoch 0 Batch 60: Train Loss = 0.3204


2023-03-23 18:27:57,464 - INFO - Epoch 0 Batch 80: Train Loss = 0.2170
2023-03-23 18:27:57,464 - INFO - Epoch 0 Batch 80: Train Loss = 0.2170
2023-03-23 18:27:57,464 - INFO - Epoch 0 Batch 80: Train Loss = 0.2170


Epoch 0 Batch 80: Train Loss = 0.2170


2023-03-23 18:28:06,731 - INFO - Epoch 0 Batch 100: Train Loss = 0.2609
2023-03-23 18:28:06,731 - INFO - Epoch 0 Batch 100: Train Loss = 0.2609
2023-03-23 18:28:06,731 - INFO - Epoch 0 Batch 100: Train Loss = 0.2609


Epoch 0 Batch 100: Train Loss = 0.2609


In [64]:
if target_dataset == 'TJ':    
    file_name = './model/pretrained-challenge-front-fill-teacher-2covid'
elif target_dataset == 'HM':
    file_name = './model/pretrained-challenge-front-fill-teacher-2spain'
    
checkpoint = torch.load(file_name, \
                        map_location=torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu') )
save_epoch = checkpoint['epoch']
print("last saved model is in epoch {}".format(save_epoch))
logger.info("last saved model is in epoch {}".format(save_epoch))
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()


2023-08-06 12:57:17,957 - INFO - last saved model is in epoch 115
2023-08-06 12:57:17,957 - INFO - last saved model is in epoch 115


last saved model is in epoch 115


distcare_teacher(
  (PositionalEncoding): PositionalEncoding(
    (dropout): Dropout(p=0, inplace=False)
  )
  (GRUs): ModuleList(
    (0): GRU(1, 32, batch_first=True)
    (1): GRU(1, 32, batch_first=True)
    (2): GRU(1, 32, batch_first=True)
    (3): GRU(1, 32, batch_first=True)
    (4): GRU(1, 32, batch_first=True)
    (5): GRU(1, 32, batch_first=True)
    (6): GRU(1, 32, batch_first=True)
    (7): GRU(1, 32, batch_first=True)
    (8): GRU(1, 32, batch_first=True)
    (9): GRU(1, 32, batch_first=True)
    (10): GRU(1, 32, batch_first=True)
    (11): GRU(1, 32, batch_first=True)
    (12): GRU(1, 32, batch_first=True)
    (13): GRU(1, 32, batch_first=True)
    (14): GRU(1, 32, batch_first=True)
    (15): GRU(1, 32, batch_first=True)
    (16): GRU(1, 32, batch_first=True)
    (17): GRU(1, 32, batch_first=True)
    (18): GRU(1, 32, batch_first=True)
    (19): GRU(1, 32, batch_first=True)
    (20): GRU(1, 32, batch_first=True)
    (21): GRU(1, 32, batch_first=True)
    (22): GRU(1, 32, 

In [65]:
batch_loss = []
y_true = []
y_pred = []
pad_token = np.zeros(34)
with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_mask_x, batch_lens) in enumerate(batch_iter(test_x, test_y, test_mask_x, test_x_len, batch_size, shuffle=True)):  
        optimizer.zero_grad()
        batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
        batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
        batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
        batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)

        masks = length_to_mask(batch_lens).unsqueeze(-1).float()

        opt, decov_loss, emb = model(batch_x, batch_lens)

        BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))
#             REC_Loss = F.mse_loss(masks * recon, masks * batch_x, reduction='mean').to(device)

        model_loss =  BCE_Loss 

        loss = model_loss
        batch_loss.append(loss.cpu().detach().numpy())
        if step % 20 == 0:
            print('Batch %d: Test Loss = %.4f'%(step, loss.cpu().detach().numpy()))
            logger.info('Batch %d: Test Loss = %.4f'%(step, loss.cpu().detach().numpy()))
        y_pred += list(opt.cpu().detach().numpy().flatten())
        y_true += list(batch_y.cpu().numpy().flatten())

print("\n==>Predicting on test")
print('Test Loss = %.4f'%(np.mean(np.array(batch_loss))))
logger.info("\n==>Predicting on test")
logger.info('Test Loss = %.4f'%(np.mean(np.array(batch_loss))))
y_pred = np.array(y_pred)
y_pred = np.stack([1 - y_pred, y_pred], axis=1)
test_res = metrics.print_metrics_binary(y_true, y_pred)

/home/zzj/anaconda3/envs/distcare/lib/python3.7/site-packages/ipykernel_launcher.py:163: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
2023-08-06 12:57:18,588 - INFO - Batch 0: Test Loss = 0.1019
2023-08-06 12:57:18,588 - INFO - Batch 0: Test Loss = 0.1019


Batch 0: Test Loss = 0.1019


2023-08-06 12:57:28,675 - INFO - 
==>Predicting on test
2023-08-06 12:57:28,675 - INFO - 
==>Predicting on test
2023-08-06 12:57:28,677 - INFO - Test Loss = 0.1250
2023-08-06 12:57:28,677 - INFO - Test Loss = 0.1250



==>Predicting on test
Test Loss = 0.1250
confusion matrix:
[[3696   40]
 [ 125  172]]
accuracy = 0.9590875506401062
precision class 0 = 0.9672860503196716
precision class 1 = 0.8113207817077637
recall class 0 = 0.9892933368682861
recall class 1 = 0.5791245698928833
AUC of ROC = 0.9431322504127642
AUC of PRC = 0.7653587745674763
min(+P, Se) = 0.7037037037037037
f1_score = 0.6758349736086785


#### Student Model

In [66]:
class distcare_student(nn.Module):
    def __init__(self, input_dim, hidden_dim, d_model,  MHD_num_head, d_ff, output_dim, keep_prob=0.5):
        super(distcare_student, self).__init__()

        # hyperparameters
        self.input_dim = input_dim  
        self.hidden_dim = hidden_dim  # d_model
        self.d_model = d_model
        self.MHD_num_head = MHD_num_head
        self.d_ff = d_ff
        self.output_dim = output_dim
        self.keep_prob = keep_prob

        # layers
        self.PositionalEncoding = PositionalEncoding(self.d_model, dropout = 0, max_len = 400)

        self.GRUs = clones(nn.GRU(1, self.hidden_dim, batch_first = True), self.input_dim)
        self.generalGRU =  nn.GRU(1, self.hidden_dim, batch_first = True)
        self.LastStepAttentions = clones(SingleAttention(self.hidden_dim, 8, attention_type='concat', demographic_dim=12, time_aware=True, use_demographic=False),self.input_dim)
        
        self.FinalAttentionQKV = FinalAttentionQKV(self.hidden_dim, self.hidden_dim, attention_type='mul',dropout = 1 - self.keep_prob)

        self.MultiHeadedAttention = MultiHeadedAttention(self.MHD_num_head, self.d_model,dropout = 1 - self.keep_prob)
        self.SublayerConnection = SublayerConnection(self.d_model, dropout = 1 - self.keep_prob)

        self.PositionwiseFeedForward = PositionwiseFeedForward(self.d_model, self.d_ff, dropout=0.1)

        self.demo_proj_main = nn.Linear(12, self.hidden_dim)
        self.demo_proj = nn.Linear(12, self.hidden_dim)
        self.output = nn.Linear(self.hidden_dim, self.output_dim)

        self.dropout = nn.Dropout(p = 1 - self.keep_prob)
        self.FC_embed = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.tanh=nn.Tanh()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()
        self.to_MMD = nn.Linear(self.hidden_dim, 1)

    def forward(self, input, input_diff, lens, tar, train):
        lens = lens.to('cpu')
        tar_x, tar_lens = tar
        # input shape [batch_size, timestep, feature_dim]
#         demo_main = self.tanh(self.demo_proj_main(demo_input)).unsqueeze(1)# b hidden_dim
        
        batch_size = input.size(0)
        time_step = input.size(1)
        feature_dim = input.size(2)
        feature_dim_diff = input_diff.size(2)
        assert(feature_dim == self.input_dim)# input Tensor : 256 * 48 * 76
        assert(self.d_model % self.MHD_num_head == 0)

        # Initialization
        #cur_hs = Variable(torch.zeros(batch_size, self.hidden_dim).unsqueeze(0))

        # forward
        # GRU_embeded_input = self.GRUs[0](input[:,:,0].unsqueeze(-1), Variable(torch.zeros(batch_size, self.hidden_dim).unsqueeze(0)).to(device))[0] # b t h
        # Attention_embeded_input = self.LastStepAttentions[0](GRU_embeded_input)[0].unsqueeze(1)# b 1 h
        # for i in range(feature_dim-1):
        #     embeded_input = self.GRUs[i+1](input[:,:,i+1].unsqueeze(-1), Variable(torch.zeros(batch_size, self.hidden_dim).unsqueeze(0)).to(device))[0] # b 1 h
        #     embeded_input = self.LastStepAttentions[i+1](embeded_input)[0].unsqueeze(1)# b 1 h
        #     Attention_embeded_input = torch.cat((Attention_embeded_input, embeded_input), 1)# b i h

        # Attention_embeded_input = torch.cat((Attention_embeded_input, demo_main), 1)# b i+1 h
        # posi_input = self.dropout(Attention_embeded_input) # batch_size * d_input+1 * hidden_dim

#         input = pack_padded_sequence(input, lens, batch_first=True)
        
        GRU_embeded_input = self.GRUs[0](pack_padded_sequence(input[:,:,0].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
#         print(GRU_embeded_input.shape)
        for i in range(feature_dim-1):
            embeded_input = self.GRUs[i+1](pack_padded_sequence(input[:,:,i+1].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
            GRU_embeded_input = torch.cat((GRU_embeded_input, embeded_input), 1)

#         GRU_embeded_input = torch.cat((GRU_embeded_input, demo_main), 1)# b i+1 h
        General_GRU_embeded_input = self.generalGRU(pack_padded_sequence(input_diff[:,:,0].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
        for i in range(1,feature_dim_diff):
            general_embeded_input = self.generalGRU(pack_padded_sequence(input_diff[:,:,i].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
            General_GRU_embeded_input = torch.cat((General_GRU_embeded_input,general_embeded_input), 1)
        
#         posi_input = self.dropout(GRU_embeded_input) # batch_size * d_input * hidden_dim
        posi_input = self.dropout(torch.cat((GRU_embeded_input, General_GRU_embeded_input), 1)) # batch_size * d_input * hidden_dim
    
        # cul tar loss
        if train:
            tar_lens = tar_lens.to('cpu')
            GRU_tar_input = self.GRUs[0](pack_padded_sequence(tar_x[:,:,0].unsqueeze(-1), tar_lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
            for i in range(feature_dim-1):
                tar_input = self.GRUs[i+1](pack_padded_sequence(tar_x[:,:,i+1].unsqueeze(-1), tar_lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
                GRU_tar_input = torch.cat((GRU_tar_input, tar_input), 1)
            GRU_embeded_output = self.to_MMD(GRU_embeded_input)
            GRU_tar_output = self.to_MMD(GRU_tar_input)
        else:
            GRU_embeded_output = []
            GRU_tar_output = []
        
        #mask = subsequent_mask(time_step).to(device) # 1 t t 下三角 N to 1任务不用mask
        contexts = self.SublayerConnection(posi_input, lambda x: self.MultiHeadedAttention(posi_input, posi_input, posi_input, None))# # batch_size * d_input * hidden_dim
    
        DeCov_loss = contexts[1]
        contexts = contexts[0]

        contexts = self.SublayerConnection(contexts, lambda x: self.PositionwiseFeedForward(contexts))[0]# # batch_size * d_input * hidden_dim
        #contexts = contexts.view(batch_size, feature_dim * self.hidden_dim)#
        # contexts = torch.matmul(self.Wproj, contexts) + self.bproj
        # contexts = contexts.squeeze()
        # demo_key = self.demo_proj(demo_input)# b hidden_dim
        # demo_key = self.relu(demo_key)
        # input_dim_scores = torch.matmul(contexts, demo_key.unsqueeze(-1)).squeeze() # b i
        # input_dim_scores = self.dropout(self.sigmoid(input_dim_scores)).unsqueeze(1)# b i
        
        # weighted_contexts = torch.matmul(input_dim_scores, contexts).squeeze()
#         print(contexts.shape)

        weighted_contexts = self.FinalAttentionQKV(contexts)[0]
        output_embed = self.FC_embed(weighted_contexts)
        output = self.output(self.dropout(weighted_contexts))# b 1
        output = self.sigmoid(output)
#         print(weighted_contexts.shape)
          
        return output, DeCov_loss, output_embed, [GRU_embeded_output, GRU_tar_output]
    #, self.MultiHeadedAttention.attn




In [67]:
logger.info("load target data")
if target_dataset == 'TJ':
    data_path = './data/Tongji/'
    tar_all_x = pickle.load(open(data_path + 'x.pkl', 'rb'))
    tar_all_y = pickle.load(open(data_path + 'y.pkl', 'rb'))
    tar_all_time = pickle.load(open(data_path + 'y.pkl', 'rb'))
    tar_all_x_len = [len(i) for i in tar_all_x]

    for i in range(len(tar_all_time)):
        for j in range(len(tar_all_time[i])):
            tar_all_time[i][j] = tar_all_time[i][j][-1]
            tar_all_y[i][j] = tar_all_y[i][j][0]

    tar_subset_idx = [2, 3, 4, 9, 13, 14, 26, 27, 30, 32, 34, 38, 39, 41, 52, 53, 66, 74]
    tar_other_idx = list(range(75))
    for i in tar_subset_idx:
        tar_other_idx.remove(i)
    for i in range(len(tar_all_x)):
        cur = np.array(tar_all_x[i], dtype=float)
        cur_subset = cur[:, tar_subset_idx]
        cur_other = cur[:, tar_other_idx]
    #     tar_all_x[i] = np.concatenate((cur_subset, cur_other), axis=1).tolist()
        tar_all_x[i] = cur_subset
elif target_dataset == 'HM':
    data_path = './data/CDSL/'
    tar_all_x = pickle.load(open(data_path + 'x.pkl', 'rb'))
    tar_all_y = pickle.load(open(data_path + 'y.pkl', 'rb'))
    tar_all_time = pickle.load(open(data_path + 'y.pkl', 'rb'))
    tar_all_x_len = [len(i) for i in tar_all_x]

    for i in range(len(tar_all_time)):
        for j in range(len(tar_all_time[i])):
            tar_all_time[i][j] = tar_all_time[i][j][-1]
            tar_all_y[i][j] = tar_all_y[i][j][0]

    tar_subset_idx = [5, 6, 4, 2, 3, 48, 79, 76, 87, 25, 30, 31, 18, 43, 58, 66, 40, 57, 23, 92, 50, 54, 91, 60, 39, 81]
    tar_other_idx = list(range(99))
    for i in tar_subset_idx:
        tar_other_idx.remove(i)
    for i in range(len(tar_all_x)):
        cur = np.array(tar_all_x[i], dtype=float)
        cur_subset = cur[:, tar_subset_idx]
        cur_other = cur[:, tar_other_idx]
    #     tar_all_x[i] = np.concatenate((cur_subset, cur_other), axis=1).tolist()
        tar_all_x[i] = cur_subset
    
print(tar_all_x[0])
print(len(tar_all_x[0][0]))
print(len(tar_all_x))
logger.info(tar_all_x[0])
logger.info(len(tar_all_x[0][0]))
logger.info(len(tar_all_x))

assert(subset_cnt == len(tar_subset_idx))

examples = []
for idx in range(len(tar_all_x)):
    examples.append((tar_all_x[idx], tar_all_y[idx], tar_all_time[idx], tar_all_x_len[idx]))
examples = sorted(examples, key=lambda e: len(e[0]), reverse=True)
tar_all_x = [e[0] for e in examples]
tar_all_y = [e[1] for e in examples]
tar_all_time = [e[2] for e in examples]
tar_all_x_len = [e[3] for e in examples]

2023-08-06 12:57:28,751 - INFO - load target data
2023-08-06 12:57:28,751 - INFO - load target data
2023-08-06 12:57:31,944 - INFO - [[ 0.35672857  0.36219378 -0.04155072 -0.39134566  0.76306464 -0.05394446
   0.23950892 -0.09317341  0.27449751 -0.08003117 -0.11276048 -0.19232824
  -0.32689733 -0.30210297 -0.1920395  -0.0936725  -0.05866173 -0.02507609
  -0.25397049 -0.38426823  0.1113217   0.10224702 -0.17622869 -0.22210885
  -0.04331355 -0.14866188]
 [ 2.89139623 -0.16505646 -0.7197426  -0.5274138   0.76306464 -0.05394446
   0.23950892 -0.09317341  0.27449751 -0.08003117 -0.11276048 -0.19232824
  -0.32689733 -0.30210297 -0.1920395  -0.0936725  -0.05866173 -0.02507609
  -0.25397049 -0.38426823  0.1113217   0.10224702 -0.17622869 -0.22210885
  -0.04331355 -0.14866188]
 [-0.72955757  0.88944403  1.54089701 -1.13972045  0.01202093 -0.05394446
   0.23950892 -0.09317341  0.27449751 -0.08003117 -0.11276048 -0.19232824
  -0.32689733 -0.30210297 -0.1920395  -0.0936725  -0.05866173 -0.02507609

2023-08-06 12:57:31,944 - INFO - [[ 0.35672857  0.36219378 -0.04155072 -0.39134566  0.76306464 -0.05394446
   0.23950892 -0.09317341  0.27449751 -0.08003117 -0.11276048 -0.19232824
  -0.32689733 -0.30210297 -0.1920395  -0.0936725  -0.05866173 -0.02507609
  -0.25397049 -0.38426823  0.1113217   0.10224702 -0.17622869 -0.22210885
  -0.04331355 -0.14866188]
 [ 2.89139623 -0.16505646 -0.7197426  -0.5274138   0.76306464 -0.05394446
   0.23950892 -0.09317341  0.27449751 -0.08003117 -0.11276048 -0.19232824
  -0.32689733 -0.30210297 -0.1920395  -0.0936725  -0.05866173 -0.02507609
  -0.25397049 -0.38426823  0.1113217   0.10224702 -0.17622869 -0.22210885
  -0.04331355 -0.14866188]
 [-0.72955757  0.88944403  1.54089701 -1.13972045  0.01202093 -0.05394446
   0.23950892 -0.09317341  0.27449751 -0.08003117 -0.11276048 -0.19232824
  -0.32689733 -0.30210297 -0.1920395  -0.0936725  -0.05866173 -0.02507609
  -0.25397049 -0.38426823  0.1113217   0.10224702 -0.17622869 -0.22210885
  -0.04331355 -0.14866188

2023-08-06 12:57:31,967 - INFO - 26
2023-08-06 12:57:31,967 - INFO - 26
2023-08-06 12:57:31,969 - INFO - 4255
2023-08-06 12:57:31,969 - INFO - 4255


[[ 0.35672857  0.36219378 -0.04155072 -0.39134566  0.76306464 -0.05394446
   0.23950892 -0.09317341  0.27449751 -0.08003117 -0.11276048 -0.19232824
  -0.32689733 -0.30210297 -0.1920395  -0.0936725  -0.05866173 -0.02507609
  -0.25397049 -0.38426823  0.1113217   0.10224702 -0.17622869 -0.22210885
  -0.04331355 -0.14866188]
 [ 2.89139623 -0.16505646 -0.7197426  -0.5274138   0.76306464 -0.05394446
   0.23950892 -0.09317341  0.27449751 -0.08003117 -0.11276048 -0.19232824
  -0.32689733 -0.30210297 -0.1920395  -0.0936725  -0.05866173 -0.02507609
  -0.25397049 -0.38426823  0.1113217   0.10224702 -0.17622869 -0.22210885
  -0.04331355 -0.14866188]
 [-0.72955757  0.88944403  1.54089701 -1.13972045  0.01202093 -0.05394446
   0.23950892 -0.09317341  0.27449751 -0.08003117 -0.11276048 -0.19232824
  -0.32689733 -0.30210297 -0.1920395  -0.0936725  -0.05866173 -0.02507609
  -0.25397049 -0.38426823  0.1113217   0.10224702 -0.17622869 -0.22210885
  -0.04331355 -0.14866188]
 [-0.72955757  0.36219378  1.54

In [68]:
epochs = 150
batch_size = 256
input_dim = subset_cnt
hidden_dim = 32
d_model = 32
MHD_num_head = 4
d_ff = 64
output_dim = 1

model_student = distcare_student(input_dim = input_dim, hidden_dim = hidden_dim, d_model=d_model, MHD_num_head=MHD_num_head, d_ff=d_ff, output_dim = output_dim).to(device)
# input_dim, d_model, d_k, d_v, MHD_num_head, d_ff, output_dim
optimizer_student = torch.optim.Adam(model_student.parameters(), lr=1e-3)



In [69]:
#Generate Teacher model embedding
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

train_teacher_emb = []
batch_loss = []
y_true = []
y_pred = []
pad_token = np.zeros(34)
with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_mask_x, batch_lens) in enumerate(batch_iter(train_x, train_y, train_mask_x, train_x_len, batch_size, shuffle=False)):  
        optimizer.zero_grad()
        batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
        batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
        batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
        batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)

        masks = length_to_mask(batch_lens).unsqueeze(-1).float()

        opt, decov_loss, emb = model(batch_x, batch_lens)
        train_teacher_emb.append(emb.cpu().detach().numpy())

        BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))
#             REC_Loss = F.mse_loss(masks * recon, masks * batch_x, reduction='mean').to(device)

        model_loss =  BCE_Loss 
        if step % 20 == 0:
            print('Batch %d: Test Loss = %.4f'%(step, model_loss.cpu().detach().numpy()))
            logger.info('Batch %d: Test Loss = %.4f'%(step, model_loss.cpu().detach().numpy()))

/home/zzj/anaconda3/envs/distcare/lib/python3.7/site-packages/ipykernel_launcher.py:163: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
2023-08-06 12:57:32,505 - INFO - Batch 0: Test Loss = 0.1170
2023-08-06 12:57:32,505 - INFO - Batch 0: Test Loss = 0.1170


Batch 0: Test Loss = 0.1170


2023-08-06 12:57:44,256 - INFO - Batch 20: Test Loss = 0.1377
2023-08-06 12:57:44,256 - INFO - Batch 20: Test Loss = 0.1377


Batch 20: Test Loss = 0.1377


2023-08-06 12:57:52,431 - INFO - Batch 40: Test Loss = 0.1260
2023-08-06 12:57:52,431 - INFO - Batch 40: Test Loss = 0.1260


...

Batch 80: Test Loss = 0.0535


2023-08-06 12:58:22,683 - INFO - Batch 100: Test Loss = 0.0598
2023-08-06 12:58:22,683 - INFO - Batch 100: Test Loss = 0.0598


Batch 100: Test Loss = 0.0598


2023-08-06 12:58:33,345 - INFO - Batch 120: Test Loss = 0.0739
2023-08-06 12:58:33,345 - INFO - Batch 120: Test Loss = 0.0739


Batch 120: Test Loss = 0.0739


In [70]:
tar_all_x = torch.tensor(pad_sents(tar_all_x, np.zeros(len(tar_subset_idx))), dtype=torch.float32).to(device)
tar_all_x_len = torch.tensor(tar_all_x_len, dtype=torch.float32).to(device).int()

In [71]:
class MMDLoss(nn.Module):
    def __init__(self, kernel_type='rbf', kernel_mul=2.0, kernel_num=5, fix_sigma=None, **kwargs):
        super(MMDLoss, self).__init__()
        self.kernel_num = kernel_num
        self.kernel_mul = kernel_mul
        self.fix_sigma = None
        self.kernel_type = kernel_type

    def guassian_kernel(self, source, target, kernel_mul, kernel_num, fix_sigma):
        n_samples = int(source.size()[0]) + int(target.size()[0])
        total = torch.cat([source, target], dim=0)
        total0 = total.unsqueeze(0).expand(
            int(total.size(0)), int(total.size(0)), int(total.size(1)))
        total1 = total.unsqueeze(1).expand(
            int(total.size(0)), int(total.size(0)), int(total.size(1)))
        L2_distance = ((total0-total1)**2).sum(2)
        if fix_sigma:
            bandwidth = fix_sigma
        else:
            bandwidth = torch.sum(L2_distance.data) / (n_samples**2-n_samples)
        bandwidth /= kernel_mul ** (kernel_num // 2)
        bandwidth_list = [bandwidth * (kernel_mul**i)
                          for i in range(kernel_num)]
        kernel_val = [torch.exp(-L2_distance / bandwidth_temp)
                      for bandwidth_temp in bandwidth_list]
        return sum(kernel_val)

    def linear_mmd2(self, f_of_X, f_of_Y):
        loss = 0.0
        delta = f_of_X.float().mean(0) - f_of_Y.float().mean(0)
        loss = delta.dot(delta.T)
        return loss

    def forward(self, source, target):
        if self.kernel_type == 'linear':
            return self.linear_mmd2(source, target)
        elif self.kernel_type == 'rbf':
            batch_size = int(source.size()[0])
            kernels = self.guassian_kernel(
                source, target, kernel_mul=self.kernel_mul, kernel_num=self.kernel_num, fix_sigma=self.fix_sigma)
            XX = torch.mean(kernels[:batch_size, :batch_size])
            YY = torch.mean(kernels[batch_size:, batch_size:])
            XY = torch.mean(kernels[:batch_size, batch_size:])
            YX = torch.mean(kernels[batch_size:, :batch_size])
            loss = torch.mean(XX + YY - XY - YX)
            return loss

class MultitaskLoss(nn.Module):
    def __init__(self, task_num=3):
        super(MultitaskLoss, self).__init__()
        self.task_num = task_num
        self.alpha = nn.Parameter(torch.ones((task_num)), requires_grad=True)
        self.bce = nn.BCELoss()
        self.kl = nn.KLDivLoss(reduce=True, size_average=True)
        self.tar = MMDLoss()

    def forward(self, opt_student, batch_y, emb_student, emb_teacher, tar_source, tar_tar):
        BCE_Loss = self.bce(opt_student, batch_y)
        emb_Loss = self.kl(emb_student, emb_teacher)
        tar_Loss = self.tar(source=tar_source, target=tar_tar)
        return BCE_Loss * self.alpha[0] + emb_Loss * self.alpha[1] + tar_Loss * self.alpha[2]

def get_multitask_loss(opt_student, batch_y, emb_student, emb_teacher, tar_source, tar_tar):
    mtl = MultitaskLoss(task_num=3)
    return mtl(opt_student, batch_y, emb_student, emb_teacher, tar_source, tar_tar)

In [ ]:
# Training Student
# If you don't want to train Student Model:
# - The pretrained student model is in direcrtory './model/', and can be directly loaded, 
# - Simply skip this cell and load the model to validate on Dev Dataset.

logger.info('Training Student')
teacher_flag = True
epochs = 200
total_train_loss = []
total_valid_loss = []
global_best = 0
auroc = []
auprc = []
minpse = []
history = []

pad_token = np.zeros(34)
# begin_time = time.time()
best_auroc = 0
best_auprc = 0
best_minpse = 0

if target_dataset == 'TJ':    
    data_str = 'covid'
elif target_dataset == 'HM':
    data_str = 'spain'

if teacher_flag:
    file_name = './model/pretrained-challenge-front-fill-2'+ data_str
else: 
    file_name = './model/pretrained-challenge-front-fill-2'+ data_str + '-noteacher'

for each_epoch in range(epochs):

    epoch_loss = []
    counter_batch = 0
    model_student.train()  
    model.eval()
    for step, (batch_x, batch_y, batch_mask_x, batch_lens) in enumerate(batch_iter(train_x, train_y, train_mask_x, train_x_len, batch_size, shuffle=False)):  
        optimizer_student.zero_grad()
        batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
        batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
        batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
        batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)

#        masks = length_to_mask(batch_lens).unsqueeze(-1).float()

        opt_student, decov_loss_student, emb_student, tar_result = model_student(batch_x[:,:,:subset_cnt], batch_x[:,:,subset_cnt:], batch_lens, [tar_all_x, tar_all_x_len], True)
        emb_teacher = torch.tensor(train_teacher_emb[step], dtype=torch.float32).to(device)
        # opt_teacher, decov_loss_teacher, emb_teacher = model(batch_x, batch_lens)
#         BCE_Loss = get_loss(opt_student, batch_y.unsqueeze(-1)) # b t 1
        #emb_Loss = 0.1 * get_re_loss(emb_student, emb_teacher.detach())
        if teacher_flag:
            emb_student = F.log_softmax(emb_student, dim=1)
            emb_teacher = F.softmax(emb_teacher.detach(), dim=1)
#             emb_Loss = get_kl_loss(emb_student, emb_teacher)
            
            tar_source = F.softmax(tar_result[0].squeeze(), dim=-1)
            tar_tar = F.softmax(tar_result[1].squeeze(), dim=-1)
#             shrink_indices = torch.tensor(np.random.choice(range(len(tar_tar)), len(tar_source))).to(device)
#             tar_tar = torch.index_select(tar_tar, 0, shrink_indices)
#             tar_Loss = get_kl_loss(tar_source, tar_tar)
#             logger.info(tar_tar)
#             logger.info(tar_source)
#             logger.info(tar_Loss)
            loss = get_multitask_loss(opt_student, batch_y.unsqueeze(-1), emb_student, emb_teacher, tar_source, tar_tar)
        # emb_Loss = 0.1 * get_wass_dist(emb_student, emb_teacher.detach())
#             REC_Loss = F.mse_loss(masks * recon, masks * batch_x, reduction='mean').to(device)
        else:
            loss = BCE_Loss #+ decov_loss_student

        epoch_loss.append(BCE_Loss.cpu().detach().numpy())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_student.parameters(), 20)
        optimizer_student.step()

        if step % 20 == 0:
            print('Epoch %d Batch %d: Train Loss = %.4f'%(each_epoch, step, loss.cpu().detach().numpy()))
            logger.info('Epoch %d Batch %d: Train Loss = %.4f'%(each_epoch, step, loss.cpu().detach().numpy()))

    epoch_loss = np.mean(epoch_loss)
    total_train_loss.append(epoch_loss)

    #Validation
    y_true = []
    y_pred = []
    with torch.no_grad():
        model_student.eval()
        valid_loss = []
        valid_true = []
        valid_pred = []
        for batch_x, batch_y, batch_mask_x, batch_lens in batch_iter(dev_x, dev_y, dev_mask_x, dev_x_len, batch_size):
            batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
            batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
            batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
            batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)
#            masks = length_to_mask(batch_lens).unsqueeze(-1).float()


            opt_student, decov_loss_student, emb, tar_result = model_student(batch_x[:,:,:subset_cnt], batch_x[:,:,subset_cnt:], batch_lens, [[], []], False)

            BCE_Loss = get_loss(opt_student, batch_y.unsqueeze(-1))
#                 REC_Loss = F.mse_loss(recon, batch_x, reduction='mean').to(device)

            valid_loss.append(BCE_Loss.cpu().detach().numpy())

            y_pred += list(opt_student.cpu().detach().numpy().flatten())
            y_true += list(batch_y.cpu().numpy().flatten())

        valid_loss = np.mean(valid_loss)
        total_valid_loss.append(valid_loss)
        ret = metrics.print_metrics_binary(y_true, y_pred,verbose = 0)
        history.append(ret)
        #print()

        print('Epoch %d: Loss = %.4f Valid loss = %.4f roc = %.4f'%(each_epoch, total_train_loss[-1], total_valid_loss[-1], ret['auroc']))
        metrics.print_metrics_binary(y_true, y_pred)

        cur_auroc = ret['auroc']
        if cur_auroc > best_auroc:
            best_auroc = cur_auroc
            best_auprc = ret['auprc']
            best_minpse = ret['minpse']
            state = {
                'net': model_student.state_dict(),
                'optimizer': optimizer_student.state_dict(),
                'epoch': each_epoch
            }
            torch.save(state, file_name)
            print('------------ Save best model - AUROC: %.4f ------------'%cur_auroc)
            logger.info('------------ Save best model - AUROC: %.4f ------------'%cur_auroc)

print('auroc %.4f'%(best_auroc))
print('auprc %.4f'%(best_auprc))
print('minpse %.4f'%(best_minpse)) 
logger.info('auroc %.4f'%(best_auroc))
logger.info('auprc %.4f'%(best_auprc))
logger.info('minpse %.4f'%(best_minpse))

2023-08-04 03:12:58,561 - INFO - Training Student
/home/zzj/anaconda3/envs/distcare/lib/python3.7/site-packages/ipykernel_launcher.py:163: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/zzj/anaconda3/envs/distcare/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
/home/zzj/anaconda3/envs/distcare/lib/python3.7/site-packages/torch/nn/functional.py:2905: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
2023-08-04 03:12:59,636 - INFO - Epoch 0 Batch 0: Train Loss = 0.

Epoch 0 Batch 0: Train Loss = 0.7524


2023-08-04 03:13:15,872 - INFO - Epoch 0 Batch 20: Train Loss = 0.3767


Epoch 0 Batch 20: Train Loss = 0.3767


2023-08-04 03:13:32,027 - INFO - Epoch 0 Batch 40: Train Loss = 0.3694


...

/home/zzj/Distcare-plus/utils/metrics.py:23: RuntimeWarning: invalid value encountered in float_scalars
  prec1 = cf[1][1] / (cf[1][1] + cf[0][1])
/home/zzj/Distcare-plus/utils/metrics.py:23: RuntimeWarning: invalid value encountered in float_scalars
  prec1 = cf[1][1] / (cf[1][1] + cf[0][1])
2023-08-04 03:14:46,419 - INFO - ------------ Save best model - AUROC: 0.7168 ------------


Epoch 0: Loss = 0.0091 Valid loss = 0.2593 roc = 0.7168
confusion matrix:
[[3742    0]
 [ 292    0]]
accuracy = 0.9276152849197388
precision class 0 = 0.9276152849197388
precision class 1 = nan
recall class 0 = 1.0
recall class 1 = 0.0
AUC of ROC = 0.7168164229808981
AUC of PRC = 0.16501767157534003
min(+P, Se) = 0.2191780821917808
f1_score = nan
------------ Save best model - AUROC: 0.7168 ------------


/home/zzj/anaconda3/envs/distcare/lib/python3.7/site-packages/ipykernel_launcher.py:163: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/zzj/anaconda3/envs/distcare/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
/home/zzj/anaconda3/envs/distcare/lib/python3.7/site-packages/torch/nn/functional.py:2905: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
2023-08-04 03:14:47,239 - INFO - Epoch 1 Batch 0: Train Loss = 0.3436


Epoch 1 Batch 0: Train Loss = 0.3436


2023-08-04 03:15:03,579 - INFO - Epoch 1 Batch 20: Train Loss = 0.3179


In [72]:
teacher_flag = True

if target_dataset == 'TJ':    
    data_str = 'covid'
elif target_dataset == 'HM':
    data_str = 'spain'

if teacher_flag:
    file_name = './model/pretrained-challenge-front-fill-2'+ data_str
else: 
    file_name = './model/pretrained-challenge-front-fill-2'+ data_str + '-noteacher'

checkpoint = torch.load(file_name, \
                        map_location=torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu') )
save_epoch = checkpoint['epoch']
print("last saved model is in epoch {}".format(save_epoch))
logger.info("last saved model is in epoch {}".format(save_epoch))
model_student.load_state_dict(checkpoint['net'])
optimizer_student.load_state_dict(checkpoint['optimizer'])
model_student.eval()

2023-08-06 12:58:38,307 - INFO - last saved model is in epoch 154
2023-08-06 12:58:38,307 - INFO - last saved model is in epoch 154


last saved model is in epoch 154


distcare_student(
  (PositionalEncoding): PositionalEncoding(
    (dropout): Dropout(p=0, inplace=False)
  )
  (GRUs): ModuleList(
    (0): GRU(1, 32, batch_first=True)
    (1): GRU(1, 32, batch_first=True)
    (2): GRU(1, 32, batch_first=True)
    (3): GRU(1, 32, batch_first=True)
    (4): GRU(1, 32, batch_first=True)
    (5): GRU(1, 32, batch_first=True)
    (6): GRU(1, 32, batch_first=True)
    (7): GRU(1, 32, batch_first=True)
    (8): GRU(1, 32, batch_first=True)
    (9): GRU(1, 32, batch_first=True)
    (10): GRU(1, 32, batch_first=True)
    (11): GRU(1, 32, batch_first=True)
    (12): GRU(1, 32, batch_first=True)
    (13): GRU(1, 32, batch_first=True)
    (14): GRU(1, 32, batch_first=True)
    (15): GRU(1, 32, batch_first=True)
    (16): GRU(1, 32, batch_first=True)
    (17): GRU(1, 32, batch_first=True)
    (18): GRU(1, 32, batch_first=True)
    (19): GRU(1, 32, batch_first=True)
    (20): GRU(1, 32, batch_first=True)
    (21): GRU(1, 32, batch_first=True)
    (22): GRU(1, 32, 

In [73]:
batch_loss = []
y_true = []
y_pred = []
with torch.no_grad():
    model_student.eval()
    for step, (batch_x, batch_y, batch_mask_x, batch_lens) in enumerate(batch_iter(test_x, test_y, test_mask_x, test_x_len, batch_size, shuffle=True)):  
        optimizer_student.zero_grad()
        batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
        batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
        batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
        batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)

        masks = length_to_mask(batch_lens).unsqueeze(-1).float()

        opt, decov_loss, emb, tar_result = model_student(batch_x[:,:,:subset_cnt], batch_x[:,:,subset_cnt:], batch_lens, [[], []], False)

        BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))
#             REC_Loss = F.mse_loss(masks * recon, masks * batch_x, reduction='mean').to(device)

        model_loss =  BCE_Loss 

        loss = model_loss
        batch_loss.append(loss.cpu().detach().numpy())
        if step % 20 == 0:
            print('Batch %d: Test Loss = %.4f'%(step, loss.cpu().detach().numpy()))
            logger.info('Batch %d: Test Loss = %.4f'%(step, loss.cpu().detach().numpy()))
        y_pred += list(opt.cpu().detach().numpy().flatten())
        y_true += list(batch_y.cpu().numpy().flatten())

print("\n==>Predicting on test")
print('Test Loss = %.4f'%(np.mean(np.array(batch_loss))))
logger.info("\n==>Predicting on test")
logger.info('Test Loss = %.4f'%(np.mean(np.array(batch_loss))))
y_pred = np.array(y_pred)
y_pred = np.stack([1 - y_pred, y_pred], axis=1)
test_res = metrics.print_metrics_binary(y_true, y_pred)

/home/zzj/anaconda3/envs/distcare/lib/python3.7/site-packages/ipykernel_launcher.py:163: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
2023-08-06 12:58:38,817 - INFO - Batch 0: Test Loss = 0.1631
2023-08-06 12:58:38,817 - INFO - Batch 0: Test Loss = 0.1631


Batch 0: Test Loss = 0.1631


2023-08-06 12:58:46,155 - INFO - 
==>Predicting on test
2023-08-06 12:58:46,155 - INFO - 
==>Predicting on test
2023-08-06 12:58:46,196 - INFO - Test Loss = 0.1192
2023-08-06 12:58:46,196 - INFO - Test Loss = 0.1192



==>Predicting on test
Test Loss = 0.1192
confusion matrix:
[[3683   53]
 [ 105  192]]
accuracy = 0.9608232378959656
precision class 0 = 0.9722808599472046
precision class 1 = 0.7836734652519226
recall class 0 = 0.9858136773109436
recall class 1 = 0.6464646458625793
AUC of ROC = 0.951996769983922
AUC of PRC = 0.7766793154339517
min(+P, Se) = 0.6835016835016835
f1_score = 0.7084870828191263


### Transfer Target Dataset & Model

In [74]:
logger.info("Transfer Target Dataset & Model")
# if target_dataset == 'TJ':
#     data_path = './data/Tongji/'
#     all_x = pickle.load(open(data_path + 'x.dat', 'rb'))
#     all_y = pickle.load(open(data_path + 'y.dat', 'rb'))
#     all_time = pickle.load(open(data_path + 'time_all.dat', 'rb'))
#     all_x_len = [len(i) for i in all_x]

#     tar_subset_idx = [0,1,2,7,11,12,24,25,28,30,32,36,37,39,50,51,65,73]
#     tar_other_idx = list(range(74))
#     for i in tar_subset_idx:
#         tar_other_idx.remove(i)
#     for i in range(len(all_x)):
#         cur = np.array(all_x[i], dtype=float)
#         cur_subset = cur[:, tar_subset_idx]
#         cur_other = cur[:, tar_other_idx]
#         all_x[i] = np.concatenate((cur_subset, cur_other), axis=1).tolist()
#     print(all_x[0])
#     print(len(all_x[0][0]))
#     logger.info(all_x[0])
#     logger.info(len(all_x[0][0]))
    
# elif target_dataset == 'HM':
#     data_path = './data/Spain/'
#     all_x = pickle.load(open(data_path + 'x.dat', 'rb'))
#     all_y = pickle.load(open(data_path + 'y.dat', 'rb'))
#     all_time = pickle.load(open(data_path + 'time_all.dat', 'rb'))
#     all_x_len = [len(i) for i in all_x]
    
#     tar_subset_idx = [39, 35, 23, 47, 55, 51, 22, 53, 25, 15, 43, 65, 1, 2, 48, 12, 26, 44, 49]
#     tar_other_idx = list(range(66))
#     for i in tar_subset_idx:
#         tar_other_idx.remove(i)
#     for i in range(len(all_x)):
#         cur = np.array(all_x[i], dtype=float)
#         cur_subset = cur[:, tar_subset_idx]
#         cur_other = cur[:, tar_other_idx]
#         all_x[i] = np.concatenate((cur_subset, cur_other), axis=1).tolist()
#     print(all_x[0])
#     print(len(all_x[0][0]))
#     print(all_y[0])
#     logger.info(all_x[0])
#     logger.info(len(all_x[0][0]))
#     logger.info(all_y[0])
if target_dataset == 'TJ':
    data_path = './data/Tongji/'
    all_x = pickle.load(open(data_path + 'x.pkl', 'rb'))
    all_y = pickle.load(open(data_path + 'y.pkl', 'rb'))
    all_time = pickle.load(open(data_path + 'y.pkl', 'rb'))
    all_x_len = [len(i) for i in all_x]

    for i in range(len(all_time)):
        for j in range(len(all_time[i])):
            all_time[i][j] = all_time[i][j][-1]
            all_y[i][j] = all_y[i][j][0]

    tar_subset_idx = [2, 3, 4, 9, 13, 14, 26, 27, 30, 32, 34, 38, 39, 41, 52, 53, 66, 74]
    tar_other_idx = list(range(75))
    for i in tar_subset_idx:
        tar_other_idx.remove(i)
    for i in range(len(all_x)):
        cur = np.array(all_x[i], dtype=float)
        cur_subset = cur[:, tar_subset_idx]
        cur_other = cur[:, tar_other_idx]
        all_x[i] = np.concatenate((cur_subset, cur_other), axis=1).tolist()
elif target_dataset == 'HM':
    data_path = './data/CDSL/'
    all_x = pickle.load(open(data_path + 'x.pkl', 'rb'))
    all_y = pickle.load(open(data_path + 'y.pkl', 'rb'))
    all_time = pickle.load(open(data_path + 'y.pkl', 'rb'))
    all_x_len = [len(i) for i in all_x]

    for i in range(len(all_time)):
        for j in range(len(all_time[i])):
            all_time[i][j] = all_time[i][j][-1]
            all_y[i][j] = all_y[i][j][0]

    tar_subset_idx = [5, 6, 4, 2, 3, 48, 79, 76, 87, 25, 30, 31, 18, 43, 58, 66, 40, 57, 23, 92, 50, 54, 91, 60, 39, 81]
    tar_other_idx = list(range(99))
    for i in tar_subset_idx:
        tar_other_idx.remove(i)
    for i in range(len(all_x)):
        cur = np.array(all_x[i], dtype=float)
        cur_subset = cur[:, tar_subset_idx]
        cur_other = cur[:, tar_other_idx]
        all_x[i] = np.concatenate((cur_subset, cur_other), axis=1).tolist()
    
print(all_x[0])
print(len(all_x[0][0]))
print(len(all_x))
logger.info(all_x[0])
logger.info(len(all_x[0][0]))
logger.info(len(all_x))

2023-08-06 12:58:46,383 - INFO - Transfer Target Dataset & Model
2023-08-06 12:58:46,383 - INFO - Transfer Target Dataset & Model
2023-08-06 12:58:52,066 - INFO - [[0.35672856748565834, 0.362193782095267, -0.0415507190085758, -0.39134565918228126, 0.7630646387621486, -0.053944463391532395, 0.23950892438003255, -0.09317341164862601, 0.27449751326076044, -0.08003116599596992, -0.11276048206854025, -0.19232823522339773, -0.32689733132001597, -0.30210296834341416, -0.1920394961218317, -0.09367250499452183, -0.058661728080839165, -0.02507608834202364, -0.2539704877679013, -0.38426823024859896, 0.1113217036858432, 0.10224702120257448, -0.17622868834829972, -0.2221088487413897, -0.04331355167616206, -0.14866187746753454, 0.0, 0.3582866239721009, -0.05840471544695842, 0.0020796716856398088, -0.017707468132584787, -0.30938285094998835, -0.008241366021706666, -0.07376437314271661, -0.23777159425714214, -0.28390371567955686, -0.4230107155991255, -0.11543298040046182, -0.23061455647984533, -0.0291

2023-08-06 12:58:52,066 - INFO - [[0.35672856748565834, 0.362193782095267, -0.0415507190085758, -0.39134565918228126, 0.7630646387621486, -0.053944463391532395, 0.23950892438003255, -0.09317341164862601, 0.27449751326076044, -0.08003116599596992, -0.11276048206854025, -0.19232823522339773, -0.32689733132001597, -0.30210296834341416, -0.1920394961218317, -0.09367250499452183, -0.058661728080839165, -0.02507608834202364, -0.2539704877679013, -0.38426823024859896, 0.1113217036858432, 0.10224702120257448, -0.17622868834829972, -0.2221088487413897, -0.04331355167616206, -0.14866187746753454, 0.0, 0.3582866239721009, -0.05840471544695842, 0.0020796716856398088, -0.017707468132584787, -0.30938285094998835, -0.008241366021706666, -0.07376437314271661, -0.23777159425714214, -0.28390371567955686, -0.4230107155991255, -0.11543298040046182, -0.23061455647984533, -0.029142770807359483, 0.02602235552654817, -0.03418385058726334, 0.021391782427413616, -0.2519677833718266, 0.03767890116355937, 0.03550

2023-08-06 12:58:52,079 - INFO - 99
2023-08-06 12:58:52,079 - INFO - 99
2023-08-06 12:58:52,081 - INFO - 4255
2023-08-06 12:58:52,081 - INFO - 4255


[[0.35672856748565834, 0.362193782095267, -0.0415507190085758, -0.39134565918228126, 0.7630646387621486, -0.053944463391532395, 0.23950892438003255, -0.09317341164862601, 0.27449751326076044, -0.08003116599596992, -0.11276048206854025, -0.19232823522339773, -0.32689733132001597, -0.30210296834341416, -0.1920394961218317, -0.09367250499452183, -0.058661728080839165, -0.02507608834202364, -0.2539704877679013, -0.38426823024859896, 0.1113217036858432, 0.10224702120257448, -0.17622868834829972, -0.2221088487413897, -0.04331355167616206, -0.14866187746753454, 0.0, 0.3582866239721009, -0.05840471544695842, 0.0020796716856398088, -0.017707468132584787, -0.30938285094998835, -0.008241366021706666, -0.07376437314271661, -0.23777159425714214, -0.28390371567955686, -0.4230107155991255, -0.11543298040046182, -0.23061455647984533, -0.029142770807359483, 0.02602235552654817, -0.03418385058726334, 0.021391782427413616, -0.2519677833718266, 0.03767890116355937, 0.03550519429120874, 0.08887182857332589

In [75]:
long_x = all_x
long_y = all_y
long_y_kfold = [each[-1] for each in all_y]
long_time = all_time

In [76]:
# def get_n2n_data(x, y, x_len):
#     length = len(x)
#     assert length == len(y)
#     assert length == len(x_len)
#     new_x = []
#     new_y = []
#     new_x_len = []
#     for i in range(length):
#         for j in range(len(x[i])):
#             new_x.append(x[i][:j+1])
#             new_y.append(y[i][j])
#             new_x_len.append(j+1)
#     return new_x, new_y, new_x_len
def get_n2n_data(x, y, x_len, outcome=None):
    length = len(x)
    assert length == len(y)
    assert length == len(outcome)
    assert length == len(x_len)
    new_x = []
    new_y = []
    new_outcome = []
    new_x_len = []
    for i in range(length):
        for j in range(len(x[i])):
            new_x.append(x[i][:j+1])
            new_y.append(y[i][j])
            new_outcome.append(outcome[i][j])
            new_x_len.append(j+1)
    return new_x, new_y, new_x_len, new_outcome

In [77]:
class distcare_target(nn.Module):
    def __init__(self, input_dim, hidden_dim, d_model,  MHD_num_head, d_ff, output_dim, keep_prob=0.5):
        super(distcare_target, self).__init__()

        # hyperparameters
        self.input_dim = input_dim  
        self.hidden_dim = hidden_dim  # d_model
        self.d_model = d_model
        self.MHD_num_head = MHD_num_head
        self.d_ff = d_ff
        self.output_dim = output_dim
        self.keep_prob = keep_prob

        # layers
        self.PositionalEncoding = PositionalEncoding(self.d_model, dropout = 0, max_len = 400)

        self.GRUs = clones(nn.GRU(1, self.hidden_dim, batch_first = True), self.input_dim)
        self.LastStepAttentions = clones(SingleAttention(self.hidden_dim, 16, attention_type='concat', demographic_dim=12, time_aware=True, use_demographic=False),self.input_dim)
        
        self.FinalAttentionQKV = FinalAttentionQKV(self.hidden_dim, self.hidden_dim, attention_type='mul',dropout = 1 - self.keep_prob)

        self.MultiHeadedAttention = MultiHeadedAttention(self.MHD_num_head, self.d_model,dropout = 1 - self.keep_prob)
        self.SublayerConnection = SublayerConnection(self.d_model, dropout = 1 - self.keep_prob)

        self.PositionwiseFeedForward = PositionwiseFeedForward(self.d_model, self.d_ff, dropout=0.1)

        self.demo_proj_main = nn.Linear(12, self.hidden_dim)
        self.demo_proj = nn.Linear(12, self.hidden_dim)
        self.output = nn.Linear(self.hidden_dim, self.output_dim)

        self.dropout = nn.Dropout(p = 1 - self.keep_prob)
        self.FC_embed = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.tanh=nn.Tanh()
        self.MLP = nn.Sequential(
            nn.Linear(self.hidden_dim, 8),
            nn.ReLU(),
            nn.Linear(8, self.output_dim)
        )
        self.MLP_outcome = nn.Sequential(
            nn.Linear(self.hidden_dim, 8),
            nn.ReLU(),
            nn.Linear(8, self.output_dim)
        )
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()

    def forward(self, input, lens):
        lens = lens.to('cpu')
        # input shape [batch_size, timestep, feature_dim]
#         demo_main = self.tanh(self.demo_proj_main(demo_input)).unsqueeze(1)# b hidden_dim
        
        batch_size = input.size(0)
        time_step = input.size(1)
        feature_dim = input.size(2)
        assert(feature_dim == self.input_dim)# input Tensor : 256 * 48 * 76
        assert(self.d_model % self.MHD_num_head == 0)

        # Initialization
        #cur_hs = Variable(torch.zeros(batch_size, self.hidden_dim).unsqueeze(0))

        # forward
        # GRU_embeded_input = self.GRUs[0](input[:,:,0].unsqueeze(-1), Variable(torch.zeros(batch_size, self.hidden_dim).unsqueeze(0)).to(device))[0] # b t h
        # Attention_embeded_input = self.LastStepAttentions[0](GRU_embeded_input)[0].unsqueeze(1)# b 1 h
        # for i in range(feature_dim-1):
        #     embeded_input = self.GRUs[i+1](input[:,:,i+1].unsqueeze(-1), Variable(torch.zeros(batch_size, self.hidden_dim).unsqueeze(0)).to(device))[0] # b 1 h
        #     embeded_input = self.LastStepAttentions[i+1](embeded_input)[0].unsqueeze(1)# b 1 h
        #     Attention_embeded_input = torch.cat((Attention_embeded_input, embeded_input), 1)# b i h

        # Attention_embeded_input = torch.cat((Attention_embeded_input, demo_main), 1)# b i+1 h
        # posi_input = self.dropout(Attention_embeded_input) # batch_size * d_input+1 * hidden_dim

#         input = pack_padded_sequence(input, lens, batch_first=True)
        
        GRU_embeded_input = self.GRUs[0](pack_padded_sequence(input[:,:,0].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
#         print(GRU_embeded_input.shape)
        for i in range(feature_dim-1):
            embeded_input = self.GRUs[i+1](pack_padded_sequence(input[:,:,i+1].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
            GRU_embeded_input = torch.cat((GRU_embeded_input, embeded_input), 1)
        

#         GRU_embeded_input = torch.cat((GRU_embeded_input, demo_main), 1)# b i+1 h
        posi_input = self.dropout(GRU_embeded_input) # batch_size * d_input * hidden_dim


        #mask = subsequent_mask(time_step).to(device) # 1 t t 下三角 N to 1任务不用mask
        contexts = self.SublayerConnection(posi_input, lambda x: self.MultiHeadedAttention(posi_input, posi_input, posi_input, None))# # batch_size * d_input * hidden_dim
    
        DeCov_loss = contexts[1]
        contexts = contexts[0]

        contexts = self.SublayerConnection(contexts, lambda x: self.PositionwiseFeedForward(contexts))[0]# # batch_size * d_input * hidden_dim
        #contexts = contexts.view(batch_size, feature_dim * self.hidden_dim)#
        # contexts = torch.matmul(self.Wproj, contexts) + self.bproj
        # contexts = contexts.squeeze()
        # demo_key = self.demo_proj(demo_input)# b hidden_dim
        # demo_key = self.relu(demo_key)
        # input_dim_scores = torch.matmul(contexts, demo_key.unsqueeze(-1)).squeeze() # b i
        # input_dim_scores = self.dropout(self.sigmoid(input_dim_scores)).unsqueeze(1)# b i
        
        # weighted_contexts = torch.matmul(input_dim_scores, contexts).squeeze()
#         print(contexts.shape)

        weighted_contexts = self.FinalAttentionQKV(contexts)[0]
        #output_embed = self.FC_embed(weighted_contexts)
        output = self.MLP(self.dropout(weighted_contexts))# b 1
        outcome = self.MLP_outcome(self.dropout(weighted_contexts))# b 1
        outcome = F.sigmoid(outcome)
        if self.output_dim != 1:
            output = F.softmax(output, dim=1)
#         print(weighted_contexts.shape)
          
        return output, DeCov_loss, weighted_contexts, outcome
    #, self.MultiHeadedAttention.attn




In [78]:
# def transfer_gru_dict(pretrain_dict, model_dict):
#     state_dict = {}
#     for k, v in pretrain_dict.items():
#         if 'GRUs' in k:
#             state_dict[k] = v
#             print("transfered weight: {}".format(k))
#             logger.info("transfered weight: {}".format(k))
#         else:
#             print("Other weight in model_dict: {}".format(k))
#             logger.info("Other weight in model_dict: {}".format(k))
#     return state_dict
def transfer_gru_dict(pretrain_dict, model_dict):
    state_dict = {}
    for k, v in model_dict.items():
        if "GRUs" in k:
            if k in pretrain_dict:
                state_dict[k] = pretrain_dict[k]
            else:
                target_k = "generalGRU."
                point_position1 = k.find('.')
                point_position2 = k.find('.', point_position1+1)
                target_k += k[point_position2+1:]
                state_dict[k] = pretrain_dict[target_k]
    return state_dict

In [79]:
if target_dataset == 'TJ':
    input_dim = 75
elif target_dataset == 'HM':
    input_dim = 99
    
cell = 'GRU'
hidden_dim = 32
d_model = 32
MHD_num_head = 4
d_ff = 64
output_dim = 1

In [80]:
def ckd_batch_iter(x, y, lens, batch_size, shuffle=False, outcome=None):
    """ Yield batches of source and target sentences reverse sorted by length (largest to smallest).
    @param data (list of (src_sent, tgt_sent)): list of tuples containing source and target sentence
    @param batch_size (int): batch size
    @param shuffle (boolean): whether to randomly shuffle the dataset
    """
    batch_num = math.ceil(len(x) / batch_size) # 向下取整
    index_array = list(range(len(x)))

    if shuffle:
        np.random.shuffle(index_array)

    for i in range(batch_num):
        indices = index_array[i * batch_size: (i + 1) * batch_size] #  fetch out all the induces
        
        examples = []
        for idx in indices:
            examples.append((x[idx], y[idx],  lens[idx], outcome[idx]))
       
        examples = sorted(examples, key=lambda e: len(e[0]), reverse=True)
    
        batch_x = [e[0] for e in examples]
        batch_y = [e[1] for e in examples]
#         batch_name = [e[2] for e in examples]
        batch_lens = [e[2] for e in examples]
        batch_outcome = [e[3] for e in examples]
       

        yield batch_x, batch_y, batch_lens, batch_outcome

In [81]:
class TargetMultitaskLoss(nn.Module):
    def __init__(self, task_num=2):
        super(TargetMultitaskLoss, self).__init__()
        self.task_num = task_num
        self.alpha = nn.Parameter(torch.ones((task_num)), requires_grad=True)
        self.mse = nn.MSELoss()
        self.bce = nn.BCELoss()

    def forward(self, opt_student, los, outcome, outcome_y):
        MSE_Loss = self.mse(opt_student, los)
        BCE_Loss = self.bce(outcome, outcome_y)
        return MSE_Loss * self.alpha[0] + BCE_Loss * self.alpha[1]

def get_target_multitask_loss(opt_student, los, outcome, outcome_y):
    mtl = TargetMultitaskLoss(task_num=2)
    return mtl(opt_student, los, outcome, outcome_y)

def reverse_los(y, los_info):
    return y * los_info["los_std"] + los_info["los_mean"]

In [82]:
los_info = pickle.load(open(data_path + 'los_info.pkl', 'rb'))
print(los_info)
logger.info(los_info)

2023-08-06 12:58:52,619 - INFO - {'los_mean': 5.363315937659429, 'los_std': 4.099244607743503, 'los_median': 4.333333333333333, 'large_los': 21.291666666666668, 'threshold': 3.7605509886094994}
2023-08-06 12:58:52,619 - INFO - {'los_mean': 5.363315937659429, 'los_std': 4.099244607743503, 'los_median': 4.333333333333333, 'large_los': 21.291666666666668, 'threshold': 3.7605509886094994}


{'los_mean': 5.363315937659429, 'los_std': 4.099244607743503, 'los_median': 4.333333333333333, 'large_los': 21.291666666666668, 'threshold': 3.7605509886094994}


In [ ]:
if target_dataset == 'TJ':
    n_splits = 5
    epochs = 150
elif target_dataset == 'HM':
    n_splits = 3
    epochs = 20

teacher_flag = True
transfer_flag = True
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

if target_dataset == 'TJ':    
    data_str = 'covid'
elif target_dataset == 'HM':
    data_str = 'spain'

if teacher_flag:
    file_name = './model/pretrained-challenge-front-fill-2'+ data_str
else: 
    file_name = './model/pretrained-challenge-front-fill-2'+ data_str + '-noteacher'


batch_size = 256

fold_count = 0
total_train_loss = []
total_valid_loss = []

global_best = 10000
mse = []
mad = []
mape = []
auroc = []
auprc = []
kappa = []
history = []

pad_token = np.zeros(input_dim)
# begin_time = time.time()

for train, test in kfold.split(long_x, long_y_kfold):
    train_x = [long_x[i] for i in train]
    train_y = [long_time[i] for i in train]
    train_outcome = [long_y[i] for i in train]
    train_x_len = [all_x_len[i] for i in train]
    #train_static = [long_static[i] for i in train]
    
    train_x, train_y, train_x_len, train_outcome = get_n2n_data(train_x, train_y, train_x_len, outcome=train_outcome)
    if len(train_x) % 256 == 1:
        print(len(train_x))
        print('wrong squeeze!')

for train, test in kfold.split(long_x, long_y_kfold):
    
    model = distcare_target(input_dim = input_dim,output_dim=output_dim, d_model=d_model, MHD_num_head=MHD_num_head, d_ff=d_ff, hidden_dim=hidden_dim).to(device)
    
    if transfer_flag:
        checkpoint = torch.load(file_name, \
                        map_location=torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu'))
        pretrain_dict = checkpoint['net']
        model_dict = model.state_dict()
        pretrain_dict = transfer_gru_dict(pretrain_dict, model_dict)
        model_dict.update(pretrain_dict)
        model.load_state_dict(model_dict)
        
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    fold_count += 1
#     print(train)

    
    train_x = [long_x[i] for i in train]
    train_y = [long_time[i] for i in train]
    train_outcome = [long_y[i] for i in train]
    train_x_len = [all_x_len[i] for i in train]
    #train_static = [long_static[i] for i in train]
    
    train_x, train_y, train_x_len, train_outcome = get_n2n_data(train_x, train_y, train_x_len, outcome=train_outcome)
    
    test_x = [long_x[i] for i in test]
    test_y = [long_time[i] for i in test]
    test_outcome = [long_y[i] for i in test]
    test_x_len = [all_x_len[i] for i in test]
    #test_static = [long_static[i] for i in test]
    
    test_x, test_y, test_x_len, test_outcome = get_n2n_data(test_x, test_y, test_x_len, outcome=test_outcome)
    
    if not os.path.exists('./model/'+data_str):
        os.mkdir('./model/'+data_str)
        
    if transfer_flag:
        target_file_name = './model/'+data_str+'/distcare-trans-'+str(n_splits)+'-fold-LOS-regression' + str(fold_count)#4114
    else:
        target_file_name = './model/'+data_str+'/distcare-no-trans-'+str(n_splits)+'-fold-LOS-regression' + str(fold_count)#4114
    
    fold_train_loss = []
    fold_valid_loss = []
    best_mse = 10000
    best_mad = 0
    best_auroc = 0
    beat_auprc = 0
    best_mape = 0
    best_kappa = 0
    
    for each_epoch in range(epochs):
       
        
        epoch_loss = []
        counter_batch = 0
        model.train()  
        
        for step, (batch_x, batch_y, batch_lens, batch_outcome) in enumerate(ckd_batch_iter(train_x, train_y, train_x_len, batch_size, shuffle=True, outcome=train_outcome)):  
            optimizer.zero_grad()
            batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
            batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
            batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
            batch_outcome = torch.tensor(batch_outcome, dtype=torch.float32).to(device)

            masks = length_to_mask(batch_lens).unsqueeze(-1).float()

            opt, decov_loss, emb, outcome = model(batch_x, batch_lens)

#             MSE_Loss = get_re_loss(opt, batch_y.unsqueeze(-1))
            pred_loss = get_target_multitask_loss(opt, batch_y.unsqueeze(-1), outcome, batch_outcome.unsqueeze(-1))

            model_loss = pred_loss + 1e7*decov_loss

            loss = model_loss

            epoch_loss.append(pred_loss.cpu().detach().numpy())
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 20)
            optimizer.step()
            
            if step % 50 == 0:
                print('Fold %d Epoch %d Batch %d: Train Loss = %.4f'%(fold_count,each_epoch, step, loss.cpu().detach().numpy()))
                logger.info('Fold %d Epoch %d Batch %d: Train Loss = %.4f'%(fold_count,each_epoch, step, loss.cpu().detach().numpy()))
            
        epoch_loss = np.mean(epoch_loss)
        fold_train_loss.append(epoch_loss)

        #Validation
        y_true = []
        y_pred = []
        y_pred_flatten = []
        y_true_flatten = []
        outcome_pred_flatten = []
        outcome_true_flatten = []
        with torch.no_grad():
            model.eval()
            valid_loss = []
            valid_true = []
            valid_pred = []
            for batch_x, batch_y, batch_lens, batch_outcome in ckd_batch_iter(test_x, test_y, test_x_len, batch_size, outcome=test_outcome):
                batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
                batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
                batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
                batch_outcome = torch.tensor(batch_outcome, dtype=torch.float32).to(device)
                masks = length_to_mask(batch_lens).unsqueeze(-1).float()
               
                opt, decov_loss, emb, outcome = model(batch_x, batch_lens)
                
#                 MSE_Loss = get_re_loss(opt, batch_y.unsqueeze(-1))
                pred_loss = get_target_multitask_loss(opt, batch_y.unsqueeze(-1), outcome, batch_outcome.unsqueeze(-1))
                
                valid_loss.append(pred_loss.cpu().detach().numpy())

                y_pred_flatten += [reverse_los(x, los_info) for x in list(opt.cpu().detach().numpy().flatten())]
                y_true_flatten += [reverse_los(x, los_info) for x in list(batch_y.cpu().numpy().flatten())]
                outcome_pred_flatten += list(outcome.cpu().detach().numpy().flatten())
                outcome_true_flatten += list(batch_outcome.cpu().numpy().flatten())
            

            valid_loss = np.mean(valid_loss)
            fold_valid_loss.append(valid_loss)
            ret = metrics.print_metrics_regression(y_true_flatten, y_pred_flatten, verbose=0)
            ret_outcome = metrics.print_metrics_binary(outcome_true_flatten, outcome_pred_flatten, verbose=0)
            history.append((ret, ret_outcome))
            #print()

            if each_epoch % 10 == 0:
                print('Fold %d, epoch %d: Loss = %.4f Valid loss = %.4f MSE = %.4f AUROC = %.4f' % (
                    fold_count, each_epoch, fold_train_loss[-1], fold_valid_loss[-1], ret['mse'], ret_outcome['auroc']), flush=True)
                logger.info('Fold %d, epoch %d: Loss = %.4f Valid loss = %.4f MSE = %.4f AUROC = %.4f' % (
                    fold_count, each_epoch, fold_train_loss[-1], fold_valid_loss[-1], ret['mse'], ret_outcome['auroc']))
                # metrics.print_metrics_regression(y_true_flatten, y_pred_flatten)
                
            cur_mse = ret['mse']
            if cur_mse < best_mse:
                print('------------ Save FOLD-BEST model - MSE: %.4f ------------' % cur_mse, flush=True)
                logger.info('------------ Save FOLD-BEST model - MSE: %.4f ------------' % cur_mse)
                metrics.print_metrics_regression(y_true_flatten, y_pred_flatten)
                best_mse = cur_mse
                best_mad = ret['mad']
                best_auroc = ret_outcome['auroc']
                best_auprc = ret_outcome['auprc']
                state = {
                    'net': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'epoch': each_epoch
                }
                torch.save(state, target_file_name + '_' + str(fold_count))

                if cur_mse < global_best:
                    global_best = cur_mse
                    state = {
                        'net': model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'epoch': each_epoch
                    }
                    torch.save(state, target_file_name)
                    print('------------ Save best model - MSE: %.4f ------------' % cur_mse, flush=True)
                    logger.info('------------ Save best model - MSE: %.4f ------------' % cur_mse)

        print('Fold %d, mse = %.4f, mad = %.4f' % (fold_count, ret['mse'], ret['mad']), flush=True)
        logger.info('Fold %d, mse = %.4f, mad = %.4f' % (fold_count, ret['mse'], ret['mad']))

    mse.append(best_mse)
    mad.append(best_mad)
    auroc.append(best_auroc)
    auprc.append(best_auprc)
    total_train_loss.append(fold_train_loss)
    total_valid_loss.append(fold_valid_loss)


print('mse %.4f(%.4f)' % (np.mean(mse), np.std(mse)))
print('mad %.4f(%.4f)' % (np.mean(mad), np.std(mad)))
print('auroc %.4f(%.4f)' % (np.mean(auroc), np.std(auroc)))
print('auprc %.4f(%.4f)' % (np.mean(auprc), np.std(auprc)))
logger.info('mse %.4f(%.4f)' % (np.mean(mse), np.std(mse)))
logger.info('mad %.4f(%.4f)' % (np.mean(mad), np.std(mad)))
logger.info('auroc %.4f(%.4f)' % (np.mean(auroc), np.std(auroc)))
logger.info('auprc %.4f(%.4f)' % (np.mean(auprc), np.std(auprc)))

/home/zzj/anaconda3/envs/distcare/lib/python3.7/site-packages/ipykernel_launcher.py:163: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/zzj/anaconda3/envs/distcare/lib/python3.7/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
2023-08-06 12:58:59,699 - INFO - Fold 1 Epoch 0 Batch 0: Train Loss = 4005767.7500
2023-08-06 12:58:59,699 - INFO - Fold 1 Epoch 0 Batch 0: Train Loss = 4005767.7500


Fold 1 Epoch 0 Batch 0: Train Loss = 4005767.7500


2023-08-06 12:59:46,581 - INFO - Fold 1 Epoch 0 Batch 50: Train Loss = 337.0926
2023-08-06 12:59:46,581 - INFO - Fold 1 Epoch 0 Batch 50: Train Loss = 337.0926


Fold 1 Epoch 0 Batch 50: Train Loss = 337.0926


2023-08-06 13:00:32,851 - INFO - Fold 1 Epoch 0 Batch 100: Train Loss = 5.0014
2023-08-06 13:00:32,851 - INFO - Fold 1 Epoch 0 Batch 100: Train Loss = 5.0014


...

Fold 2 Epoch 5 Batch 200: Train Loss = 2.1346


2023-08-06 15:25:57,875 - INFO - Fold 2 Epoch 5 Batch 250: Train Loss = 1.6137
2023-08-06 15:25:57,875 - INFO - Fold 2 Epoch 5 Batch 250: Train Loss = 1.6137


Fold 2 Epoch 5 Batch 250: Train Loss = 1.6137


2023-08-06 15:26:41,923 - INFO - Fold 2 Epoch 5 Batch 300: Train Loss = 1.3876
2023-08-06 15:26:41,923 - INFO - Fold 2 Epoch 5 Batch 300: Train Loss = 1.3876


Fold 2 Epoch 5 Batch 300: Train Loss = 1.3876


In [ ]:
print('mse %.4f(%.4f)' % (np.mean(mse), np.std(mse)))
print('mad %.4f(%.4f)' % (np.mean(mad), np.std(mad)))
print('auroc %.4f(%.4f)' % (np.mean(auroc), np.std(auroc)))
print('auprc %.4f(%.4f)' % (np.mean(auprc), np.std(auprc)))
logger.info('mse %.4f(%.4f)' % (np.mean(mse), np.std(mse)))
logger.info('mad %.4f(%.4f)' % (np.mean(mad), np.std(mad)))
logger.info('auroc %.4f(%.4f)' % (np.mean(auroc), np.std(auroc)))
logger.info('auprc %.4f(%.4f)' % (np.mean(auprc), np.std(auprc)))